# Open-Sora 2.0 Training

## モデルの訓練

訓練は3つのステージで構成:

1. 低解像度動画によるT2V（Text to Video）モデルの訓練
1. 低解像度動画によるI2V（Image to Video）モデルの訓練
1. 高解像度動画によるI2Vモデルのファインチューニング

訓練のコストの内訳は、ステージ1のコストが50%を占める:

![](image/table3.png)

訓練の効率化の戦略:

- 11B（110億）パラメータのFluxの蒸留モデルで、T2Vモデルを初期化
    - Fluxは、オープンソースT2I（Text to Image）モデル
- 高品質な動画データで、訓練効率を向上
    - PixArtから着想
    - ステップ1, 2では、大規模データセットから高品質なサブセットを抽出
    - ステップ3では、より厳しいフィルタリングでサブセットを抽出
- 低解像度動画での動きの学習
    - 高解像度動画の訓練コストは高いため、多様な動きは低解像度動画を中心に訓練
        - 128フレーム768pxの動画の訓練は、256pxの場合よりも40倍遅い
        - セルフアテンションの計算複雑度が二次関数的に増大するため
    - 出力がぼやけるため、高解像度動画の訓練で品質を向上
- ステップ2のアップスケーリング学習は、T2VではなくI2Vを採用し効率化:
    - 静止画を条件付けすることで、動きの生成に集中できるため
- 高解像度動画でのファインチューニングを短縮し、訓練を効率化

Open-Sora 2.0の訓練コストは、5~10倍低い:

![](image/fig7.png)

訓練設定は、Open-Sora 1.2に基づく:

- 目的関数はFlow Matchingを採用
- オプティマイザはAdamWを使用
    - $\beta$値は$(0.9, 0.999)$
    - $\epsilon$は$1\times 10^{-15}$
    - 重み減衰は使用しない
    - 学習率
        - ステージ1, 2
            - 最初の4万ステップは$5\times 10^{-5}$
            - 最後の4.5万ステップは$3\times 10^{-5}$
        - ステージ3
            - $1\times 10^{-5}$
    - 購買ノルムクリッピングの閾値は$1$

Flow matchingは、Stable Diffusion 3（SD3）と類似:

$$
\mathcal{L} = \mathbb{E}_{t,X_{0},X_{1}}[||f_{\theta}(X_{t},t,y)-(X_{0}-X_{1})||]
$$

- $X_0$: 動画の潜在表現
- $X_1 \sim \mathcal{N}(0,1)$: ガウスノイズ
- $t$: タイムステップ（ノイズ強度）
- $X_t=(1-t)X_0 + tX_1$: 補間された潜在表現（ノイズありの潜在表現）
- $y$: テキストや画像などの条件
- $(X_0 - X_1)$: 正解の速度（データからノイズへの変化の方向）
- $\mathbb{E}_{t, X_0, X_1}$: 条件を変えた時の平均

$t$は、対数正規分布からサンプリングし、$X_0$の形状に合わせてスケーリング：

$$
t' = \frac{at}{1 + (a-1)t}
$$

- $a$: $T\times H\times W$に比例するパラメータ
- 高解像度での長時間の画像はノイズの影響を受けやすいため
- 推論時も同様の方法が適用

マルチバケットトレーニングを採用:

- フレーム数・解像度・アスペクト比が似た動画をバケツに分けて、バケツごとにバッチサイズを調整する手法
    - メモリ不足（OOM）の回避
    - 並列計算時の訓練時間の均一化

ステージ1, 2のバッチサイズとバッチサイズとスループット:

![](image/table4.png)

ステージ3のバッチサイズとスループット（Context parallelism 4）:

![](image/table5.png)

高圧縮オートエンコーダー（DC-AE）を使用し、トレーニングコストをさらに削減

DC-AEを使用すると、動画生成に必要なトークン数が大幅に削減できる:

$$
D_{token} = D_{T} \times D_{H} \times D_{W} \times P_{T} \times P_{H} \times P_{W}
$$

- $D_{\text{token}}$: トークン・ダウンサンプル比（どのくらい小さく圧縮してトークン化するかの比率）
- $D_T, D_H, H_W$: 時間・高さ・幅の次元におけるオートエンコーダの圧縮率
- $P_T, P_H, P_W$: 生成モデルに入力する際のパッチ分割サイズ
- Hunyuan Videoの$D_{\text{token}}$は$4096$、DC-AEの$D_{\text{token}}$は$1024$

オートエンコーダーの性能は、情報ダウンサンプリング比で予測できる:

$$
D_{info} = \frac{D_{T} \times D_{H} \times D_{W} \times C_{in}}{C_{out}}
$$

- $D_{\text{info}}$: 入力データが潜在表現になるときに情報量が圧縮された比率
- $C_{\text{in}}$: 入力チャンネル数（RGBの場合$3$）
- $C_{\text{out}}$: 出力チャンネル数（潜在表現のチャンネル数）
- DC-AEのオートエンコーダは、Hunyuan Video VAEとStepVideo VAEの$D_{\text{info}}$と一致するように設計
- 具体的には出力チャンネル数を増やす

DC-AEのダウンサンプルブロックとアップサンプルブロックには、勾配伝播の問題がある:

- ピクセルシャッフルとピクセルアンシャッフルを使用した残差接続を導入

DC-AEで出力した高圧縮の潜在変数でモデルを訓練すると、動画生成の品質が低くなる:

- チャンネル数が多いと、潜在空間構造（latent space structure）の最適化が難しくなるため
- DC-AE訓練後に、第3層の潜在表現をDINOv2に合わせるための蒸留損失を適用し、最適化

DC-AEを使用した動画生成モデルの訓練:

1. 2000万件の最大33フレームの動画を、1.7万ステップで訓練
2. 200万件の最大128フレームの動画を、8000ステップで訓練

DC-AEを使用した動画生成モデルは、Hunyuan Video VAEよりも、訓練と推論で高い効率性:

![](image/fig8.png)

DC-AEは、256チャネルと128チャネルの2つを最初から訓練:

1. 再構成損失$\mathcal{L}_1$と知覚損失$\mathcal{L}_{\text{LIPIS}}$を使用して、25万ステップ訓練:
    $$
    \mathcal{L} = \mathcal{L}_{1} + 0.5\mathcal{L}_{LPIPS}
    $$
2. 敵対的損失$\mathcal{L}_{\text{adv}}$を使用して、20万ステップ訓練:
    $$
    \mathcal{L} = \mathcal{L_1} + 0.5\mathcal{L_{LPIPS}} + 0.05\mathcal{L_{\text{adv}}}
    $$

DC-AEの訓練設定:

- ローカルバッチサイズ$1$で$8$個のGPUを使用
- アスペクト比$1:1$、32フレーム、256pxの動画を使用
- オプティマイザはAdamWを使用（$\beta=(0.9, 0.999)$, $\epsilon=1\times 10^{-15}$, 重み減衰なし）
- オートエンコーダの学習率は5e-5
- ディスクリミネータの学習率は1e-4
- 勾配ノルムクリップは閾値1

## 条件付け

条件付けは、潜在変数に対して条件ベクトルとタスクの種類を示すチャンネルを連結して行う:

![](image/fig10.png)

- 左: 画像から動画を生成するタスク
- 中: 動画を延長するタスク
- 右: 最初と最後の画像の間を補完するタスク
- 連結後のチャンネル数は$2k + 1$

汎化性能を向上させるために、テキスト条件のドロップアウトや画像条件のドロップアウトを導入

推論には、Classifier-free Guidance（CFG）を採用:

$$
v_{t} = v_{\theta}(x_{t},t,\emptyset,\emptyset) + g_{img}\cdot(v_{\theta}(x_{t},t,\emptyset,img)-v_{\theta}(x_{t},t,\emptyset,\emptyset)) + g_{txt}\cdot(v_{\theta}(x_{t},t,txt,img)-v_{\theta}(x_{t},t,\emptyset,img))
$$

- $g_{\text{img}}$: 画像条件の強さを調整する係数
- $g_{\text{txt}}$: テキスト条件の強さを調整する係数

画像ガイダンスを強くすると、生成された動画にチラつき（flicker）が生じる:

- Guidance Oscillationを導入して安定化
    - 例えば、50ステップのサンプリングのうち、10ステップごとに$g_{\text{img}}$を$1$に落とす

I2Vでは動的な画像ガイダンス（dynamic image guidance）を採用し、動画の後半になるにつれて条件付けを強くする:

![](image/fig11.png)

- 横軸はフレーム数
- 縦軸は推論ステップ数
- 画像のガイダンススケールは$1$から$k$まで線形にスケーリング
- 推論が終わりになるにつれて、ガイダンススケーリングが$1$になるように減衰
- ガイダンスのデフォルト値は$g_{\text{img}}=3$, $g_{\text{txt}}=7.5$

動画の動きの強さはモーションスコアをキャプションに追加することで制御:

![](image/fig12.png)

## システム最適化

訓練は、ColossalAIを使用:

- 141GBのメモリ容量を持つH200 GPUを使用し、データ並列（DP, Data Parallelism）を導入
- 選択的アクティベーションチェックポインティング（Selective Activation Checkpointing）を導入
    - 一部の順伝播の計算結果を破棄し、必要に応じて再計算する技術でメモリ効率化


高解像度の動画を効率的に処理するために、複数の並列化技術を採用:

- AEでは、テンソル並列化（TP, Tensor Parallelism）を畳み込み層に適用
    - 入力または出力のチャネル次元で重みを分割する
- MMDiTでは、Zero Redundancy Opitimizer（ZeroDP）とコンテキスト並列（CP, Context Parallelism）を適用
    - ZeroDPは、DeepSpeedのZeRO2により勾配を分割し、重複なくGPUが保持する技術
    - CPは、動画とテキストのシーケンスを分割し、各GPUが独立してアテンション計算する技術

アクティベーションチェックポインティングは、ブロックの入力のみを保持し、それ以外は再計算する:

- ステージ1, 2では、デュアルブロックの8層とすべてのシングルブロックに適用
- ステージ3では、すべてのブロックで有効化し、CPUへのオフロードも使用
    - オフロードでは、Pinned Memoryと非同期データ転送を使用し、さらに効率化

分散訓練の場合、物理障害が発生しやすいので自動復旧機能（Auto Recovery）を導入

- InfiniBandの障害、ストレージシステムのクラッシュ、NCCLエラー
- 訓練の状態を監視し、速度低下・損失の急激な悪化・レスポンスがないかを検証
- 問題が検出された場合は、すべてのプロセスを停止し、ノードを診断
- 必要に応じて予備をデプロイし、最終チェックポイントから自動的に訓練を再開する
- GPU稼働率99%で、ダウンタイムを最小限に抑えた

CPUとGPU間のデータ転送を高速化するためにPyTorchのデータローダーを最適化:

- PyTorchデフォルトのPinned Memoryは、CUDAカーネル実行をブロックする可能性がある
- メモリバッファ（pre-allocated pinned memory buffer）を使用し、オーバーヘッドを削減
- データ転送と計算をオーバーラップさせて効率化
- グローバルなGC（ガベージコレクション）は、分散訓練ではバグが多いため手動でメモリかんり

チェックポイントの保存は、ピン留めされたメモリバッファに対して直接アクセスすることで高速化:

- ディスクへの書き込みは、C++による非同期ディスク書き込みにより訓練プロセスをブロックせずに効率化

## 実装

In [ ]:
try:
    import google.colab
    IN_COLAB = True
    !git clone https://github.com/hpcaitech/Open-Sora.git
    WORK_DIR = "/content/Open-Sora"
    %cd $WORK_DIR
except:
    IN_COLAB = False
    WORK_DIR = "/workspaces/open-sora/Open-Sora"
    %cd $WORK_DIR

In [ ]:
import logging
import os
import platform
import subprocess

DEBUG_LOG_PATH = os.path.join(WORK_DIR, "debug.log")

if os.path.exists(DEBUG_LOG_PATH):
    os.remove(DEBUG_LOG_PATH)

def custom_format(record):
    match record.levelno:
        case logging.DEBUG:
            level = "🟦"
        case logging.INFO:
            level = "🟩"
        case logging.WARNING:
            level = "🟨"
        case logging.ERROR:
            level = "🟥"
        case logging.CRITICAL:
            level = "🛑"
    return f"{level} {record.getMessage()}"

logger = logging.getLogger()

for handler in logger.handlers:
    logger.removeHandler(handler)

formatter = logging.Formatter()
formatter.format = custom_format

file_handler = logging.FileHandler(DEBUG_LOG_PATH)
file_handler.setFormatter(formatter)
logger.addHandler(file_handler)

stream_handler = logging.StreamHandler()
stream_handler.setFormatter(formatter)
logger.addHandler(stream_handler)
logger.setLevel(logging.DEBUG)

PYTHON_VERSION = platform.python_version()
logger.info(f"Python {PYTHON_VERSION}")

NVIDIA_SMI = subprocess.run("nvidia-smi", capture_output=True, text=True).stdout
logger.info(f"{NVIDIA_SMI}")

In [ ]:
if IN_COLAB:
    %pip install torch==2.5.1 torchvision==0.20.1 torchaudio==2.5.1 --index-url https://download.pytorch.org/whl/cu124

    %pip install \
        accelerate \
        av==13.1.0 \
        colossalai \
        ftfy \
        liger-kernel \
        omegaconf \
        mmengine \
        openai \
        pandas \
        pandarallel \
        pyarrow \
        tensorboard \
        wandb \
        --extra-index-url https://download.pytorch.org/whl/cu124

    %pip install flash-attn --no-build-isolation

    %pip install -e . --no-deps

In [ ]:
from dataclasses import dataclass, field
from einops import rearrange
from flash_attn import flash_attn_func as flash_attn_func_v2
from functools import partial
from huggingface_hub import PyTorchModelHubMixin
from inspect import signature
from liger_kernel.ops.rms_norm import LigerRMSNormFunction
from liger_kernel.ops.rope import LigerRopeFunction
from omegaconf import MISSING, OmegaConf
from torch import Tensor, nn
from torch.nn.modules.batchnorm import _BatchNorm
from typing import Any, Callable, Optional, Union, Tuple
import diffusers
import math
import torch
import torch.nn as nn
import torch.nn.functional as F
import argparse
from mmengine.config import Config
import ast

try:
    from flash_attn_interface import flash_attn_func as flash_attn_func_v3
    SUPPORT_FA3 = True
except:
    SUPPORT_FA3 = False

## データセット

動画ファイルは[hcpai-tech/open-sora-pexels-45k][1]にある

[1]: https://modelscope.cn/datasets/hpcai-tech/open-sora-pexels-45k

In [ ]:
# 269.53GBの動画データセットをダウンロード
# https://modelscope.cn/datasets/hpcai-tech/open-sora-pexels-45k

if False:
    !apt update && apt install git-lfs
    !git-lfs install
    !git clone "https://www.modelscope.cn/datasets/hpcai-tech/open-sora-pexels-45k.git"
    !cd open-sora-pexels-45k && \
        cat tar/pexels_45k.tar.* > pexels_45k.tar && \
        mkdir ../datasets && \
        mv pexels_45k ../datasets

重いので最小限の統計データのみをダウンロード

In [ ]:
!wget -nc https://modelscope.cn/datasets/hpcai-tech/open-sora-pexels-45k/resolve/master/pexels_45k_necessary.csv

In [ ]:
import pandas as pd

# 動画を評価後、フィルタリングしたデータセットのCSVを読み込む
pexels_45k_necessary_df = pd.read_csv("pexels_45k_necessary.csv")
print(len(pexels_45k_necessary_df)) # 45,817件
pexels_45k_necessary_df.head(5)

In [ ]:
pexels_45k_score_df = pexels_45k_necessary_df.sort_values(by=["num_frames"])
pexels_5 = pexels_45k_score_df.head(5).copy()
pexels_5['path'] = pexels_5['path'].str.replace('datasets/pexels_45k/', 'datasets/pexels_5/', regex=False)
pexels_5

In [ ]:
DATASETS_DIR = os.path.join(WORK_DIR, "datasets")
PEXELS_DATASET_DIR = os.path.join(DATASETS_DIR, "pexels_5")
os.makedirs(PEXELS_DATASET_DIR, exist_ok=True)

In [ ]:
import re

PEXELS_CSV_PATH = os.path.join(WORK_DIR, "pexels_5.csv")

if not os.path.exists(PEXELS_CSV_PATH):
    for path in pexels_5["path"]:
        match = re.search(r"/(\d+)_", path)
        video_id = match.group(1)
        video_url = f"https://www.pexels.com/download/video/{video_id}"
        video_path = os.path.join(PEXELS_DATASET_DIR, f"{video_id}.mp4")
        !wget -nc -O $video_path $video_url
        pexels_5["path"] = pexels_5["path"].replace(path, video_path)

    pexels_5.to_csv(os.path.join(WORK_DIR, "pexels_5.csv"), index=False)
else:
    pexels_5 = pd.read_csv(PEXELS_CSV_PATH)

pexels_5.head(5)

## DC-AEの訓練

[ドキュメント][1]より以下のコマンドで開始できる

```sh
torchrun --nproc_per_node 8 scripts/vae/train.py configs/vae/train/video_dc_ae.py
```

[1]: https://github.com/hpcaitech/Open-Sora/blob/main/docs/hcae.md

### CLIパーサ

In [ ]:
def parse_args(args) -> tuple[str, argparse.Namespace]:
    """
    This function parses the command line arguments.

    Returns:
        tuple[str, argparse.Namespace]: The path to the configuration file and the command line arguments.
    """
    parser = argparse.ArgumentParser()
    parser.add_argument("config", type=str, help="model config file path")
    args, unknown_args = parser.parse_known_args(args)
    return args.config, unknown_args

In [ ]:
def read_config(config_path: str) -> Config:
    """
    This function reads the configuration file.

    Args:
        config_path (str): The path to the configuration file.

    Returns:
        Config: The configuration object.
    """
    cfg = Config.fromfile(config_path)
    return cfg

In [ ]:
def auto_convert(value: str) -> int | float | bool | list | dict | None:
    """
    Automatically convert a string to the appropriate Python data type,
    including int, float, bool, list, dict, etc.

    Args:
        value (str): The string to convert.

    Returns:
        int, float, bool, list |  dict: The converted value.
    """
    # Handle empty string
    if value == "":
        return value

    # Handle None
    if value.lower() == "none":
        return None

    # Handle boolean values
    lower_value = value.lower()
    if lower_value == "true":
        return True
    elif lower_value == "false":
        return False

    # Try to convert the string to an integer or float
    try:
        # Try converting to an integer
        return int(value)
    except ValueError:
        pass

    try:
        # Try converting to a float
        return float(value)
    except ValueError:
        pass

    # Try to convert the string to a list, dict, tuple, etc.
    try:
        return ast.literal_eval(value)
    except (ValueError, SyntaxError):
        pass

    # If all attempts fail, return the original string
    return value


In [ ]:
def merge_args(cfg: Config, args: argparse.Namespace) -> Config:
    """
    This function merges the configuration file and command line arguments.

    Args:
        cfg (Config): The configuration object.
        args (argparse.Namespace): The command line arguments.

    Returns:
        Config: The configuration object.
    """
    for k, v in zip(args[::2], args[1::2]):
        assert k.startswith("--"), f"Invalid argument: {k}"
        k = k[2:].replace("-", "_")
        k_split = k.split(".")
        target = cfg
        for key in k_split[:-1]:
            assert key in cfg, f"Key {key} not found in config"
            target = target[key]
        if v.lower() == "none":
            v = None
        elif k in target:
            v_type = type(target[k])
            if v_type == bool:
                v = auto_convert(v)
            else:
                v = type(target[k])(v)
        else:
            v = auto_convert(v)
        target[k_split[-1]] = v
    return cfg

In [ ]:
def parse_configs(args) -> Config:
    """
    This function parses the configuration file and command line arguments.

    Returns:
        Config: The configuration object.
    """
    config, args = parse_args(args)
    cfg = read_config(config)
    cfg = merge_args(cfg, args)
    cfg.config_path = config

    # hard-coded for spatial compression
    if cfg.get("ae_spatial_compression", None) is not None:
        os.environ["AE_SPATIAL_COMPRESSION"] = str(cfg.ae_spatial_compression)

    return cfg

In [ ]:
# ======================================================
# 1. configs & runtime variables
# ======================================================
# == parse configs ==
# cfg = parse_configs()
# cfg = parse_configs([
#     "configs/vae/train/video_dc_ae.py",
# ])

# # ワークショップ用に一部を上書き

# cfg.bucket_config = {
#     "256px_ar1:1": {
#         8: (1.0, 1),
#         32: (0.0, 1),
#     }
# }

# cfg.epochs = 1

In [ ]:
# import gc
# import os
# import random
# import subprocess
# import warnings
# from contextlib import nullcontext
# from copy import deepcopy
# from pprint import pformat

# warnings.filterwarnings("ignore", category=FutureWarning)
# warnings.filterwarnings("ignore", category=UserWarning)
# gc.disable()


# import torch
# import torch.distributed as dist
# from colossalai.booster import Booster
# from colossalai.utils import set_seed
# from torch.profiler import ProfilerActivity, profile, schedule
# from tqdm import tqdm

# import wandb
# from opensora.acceleration.checkpoint import set_grad_checkpoint
# from opensora.acceleration.parallel_states import get_data_parallel_group
# from opensora.datasets.dataloader import prepare_dataloader
# from opensora.datasets.pin_memory_cache import PinMemoryCache
# from opensora.models.vae.losses import DiscriminatorLoss, GeneratorLoss, VAELoss
# from opensora.registry import DATASETS, MODELS, build_module
# from opensora.utils.ckpt import CheckpointIO, model_sharding, record_model_param_shape, rm_checkpoints
# # from opensora.utils.config import config_to_name, create_experiment_workspace, parse_configs
# from opensora.utils.config import config_to_name, create_experiment_workspace
# from opensora.utils.logger import create_logger
# from opensora.utils.misc import (
#     Timer,
#     all_reduce_sum,
#     create_tensorboard_writer,
#     is_log_process,
#     log_model_params,
#     to_torch_dtype,
# )
# from opensora.utils.optimizer import create_lr_scheduler, create_optimizer
# from opensora.utils.train import create_colossalai_plugin, set_lr, set_warmup_steps, setup_device, update_ema

In [ ]:
# torch.backends.cudnn.benchmark = True

# WAIT = 1
# WARMUP = 10
# ACTIVE = 20

# my_schedule = schedule(
#     wait=WAIT,  # number of warmup steps
#     warmup=WARMUP,  # number of warmup steps with profiling
#     active=ACTIVE,  # number of active steps with profiling
# )

In [ ]:
# ======================================================
# 1. configs & runtime variables
# ======================================================
# == parse configs ==
# cfg = parse_configs()
cfg = parse_configs([
    "configs/vae/train/video_dc_ae.py",
])

cfg.bucket_config = {
    "256px_ar1:1": {
        8: (1.0, 1),
        32: (0.0, 1),
    }
}
cfg.epochs = 1

cfg.pin_memory_cache_pre_alloc_numels = None

### デバイス設定

### データ型設定

In [ ]:
def to_torch_dtype(dtype: str | torch.dtype) -> torch.dtype:
    """
    Convert a string or a torch.dtype to a torch.dtype.

    Args:
        dtype (str | torch.dtype): The input dtype.

    Returns:
        torch.dtype: The converted dtype.
    """
    if isinstance(dtype, torch.dtype):
        return dtype
    elif isinstance(dtype, str):
        dtype_mapping = {
            "float64": torch.float64,
            "float32": torch.float32,
            "float16": torch.float16,
            "fp32": torch.float32,
            "fp16": torch.float16,
            "half": torch.float16,
            "bf16": torch.bfloat16,
        }
        if dtype not in dtype_mapping:
            raise ValueError(f"Unsupported dtype {dtype}")
        dtype = dtype_mapping[dtype]
        return dtype
    else:
        raise ValueError(f"Unsupported dtype {dtype}")

# == get dtype & device ==
dtype = to_torch_dtype(cfg.get("dtype", "bf16"))
dtype

### デバイス設定

In [ ]:
os.environ['RANK'] = '0'
os.environ['WORLD_SIZE'] = '1'
os.environ['MASTER_ADDR'] = 'localhost'
os.environ['MASTER_PORT'] = '12345'

In [ ]:
from colossalai.cluster import DistCoordinator
from colossalai.utils import get_current_device
from datetime import timedelta
import torch.distributed as dist

def setup_device() -> tuple[torch.device, DistCoordinator]:
    """
    Setup the device and the distributed coordinator.

    Returns:
        tuple[torch.device, DistCoordinator]: The device and the distributed coordinator.
    """
    assert torch.cuda.is_available(), "Training currently requires at least one GPU."
    # NOTE: A very large timeout is set to avoid some processes exit early
    dist.init_process_group(backend="nccl", timeout=timedelta(hours=24))
    torch.cuda.set_device(dist.get_rank() % torch.cuda.device_count())
    coordinator = DistCoordinator()
    device = get_current_device()

    return device, coordinator

try:
    device, coordinator = setup_device()
except Exception as e:
    print(e)
    pass

device, coordinator

### チェックポイントモジュール

In [ ]:
from colossalai.zero.low_level import LowLevelZeroOptimizer
from colossalai.booster import Booster
from torch.optim import Optimizer
from torch.optim.lr_scheduler import _LRScheduler

class CheckpointIO:
    def __init__(self, n_write_entries: int = 32):
        self.n_write_entries = n_write_entries
        self.writer: Optional[Any] = None
        self.pinned_state_dict: Optional[Dict[str, torch.Tensor]] = None
        self.master_pinned_state_dict: Optional[Dict[str, torch.Tensor]] = None
        self.master_writer: Optional[Any] = None

    def _sync_io(self):
        if self.writer is not None:
            self.writer.synchronize()
            self.writer = None
        if self.master_writer is not None:
            self.master_writer.synchronize()
            self.master_writer = None

    def __del__(self):
        self._sync_io()

    def _prepare_pinned_state_dict(self, ema: nn.Module, ema_shape_dict: dict):
        if self.pinned_state_dict is None and dist.get_rank() == 0:
            self.pinned_state_dict = _prepare_ema_pinned_state_dict(ema, ema_shape_dict)

    def _prepare_master_pinned_state_dict(self, model: nn.Module, optimizer: LowLevelZeroOptimizer):
        if self.master_pinned_state_dict is None and dist.get_rank() == 0:
            sd = {}
            w2m = optimizer.get_working_to_master_map()
            for n, p in model.named_parameters():
                master_p = w2m[id(p)]
                sd[n] = torch.empty(p.shape, dtype=master_p.dtype, pin_memory=True, device="cpu")
            self.master_pinned_state_dict = sd

    def save(
        self,
        booster: Booster,
        save_dir: str,
        model: nn.Module = None,
        ema: nn.Module = None,
        optimizer: Optimizer = None,
        lr_scheduler: _LRScheduler = None,
        sampler=None,
        epoch: int = None,
        step: int = None,
        global_step: int = None,
        batch_size: int = None,
        lora: bool = False,
        actual_update_step: int = None,
        ema_shape_dict: dict = None,
        async_io: bool = True,
        include_master_weights: bool = False,
    ) -> str:
        """
        Save a checkpoint.

        Args:
            booster (Booster): The Booster object.
            save_dir (str): The directory to save the checkpoint to.
            model (nn.Module): The model to save the checkpoint from.
            ema (nn.Module): The EMA model to save the checkpoint from.
            optimizer (Optimizer): The optimizer to save the checkpoint from.
            lr_scheduler (_LRScheduler): The learning rate scheduler to save the checkpoint from.
            sampler: The sampler to save the checkpoint from.
            epoch (int): The epoch of the checkpoint.
            step (int): The step of the checkpoint.
            global_step (int): The global step of the checkpoint.
            batch_size (int): The batch size of the checkpoint.
            lora (bool): Whether the model is trained with LoRA.

        Returns:
            str: The path to the saved checkpoint
        """
        self._sync_io()
        save_dir = os.path.join(save_dir, f"epoch{epoch}-global_step{actual_update_step}")
        os.environ["TENSORNVME_DEBUG_LOG"] = os.path.join(save_dir, "async_file_io.log")
        if model is not None:
            if not lora:
                os.makedirs(os.path.join(save_dir, "model"), exist_ok=True)
                booster.save_model(
                    model,
                    os.path.join(save_dir, "model"),
                    shard=True,
                    use_safetensors=True,
                    size_per_shard=4096,
                    use_async=async_io,
                )
            else:
                os.makedirs(os.path.join(save_dir, "lora"), exist_ok=True)
                booster.save_lora_as_pretrained(model, os.path.join(save_dir, "lora"))
        if optimizer is not None:
            booster.save_optimizer(
                optimizer, os.path.join(save_dir, "optimizer"), shard=True, size_per_shard=4096, use_async=async_io
            )
            if include_master_weights:
                self._prepare_master_pinned_state_dict(model, optimizer)
                master_weights_gathering(model, optimizer, self.master_pinned_state_dict)
        if lr_scheduler is not None:
            booster.save_lr_scheduler(lr_scheduler, os.path.join(save_dir, "lr_scheduler"))
        if ema is not None:
            self._prepare_pinned_state_dict(ema, ema_shape_dict)
            model_gathering(ema, ema_shape_dict, self.pinned_state_dict)
        if dist.get_rank() == 0:
            running_states = {
                "epoch": epoch,
                "step": step,
                "global_step": global_step,
                "batch_size": batch_size,
                "actual_update_step": actual_update_step,
            }
            save_json(running_states, os.path.join(save_dir, "running_states.json"))

            if ema is not None:
                if async_io:
                    self.writer = async_save(os.path.join(save_dir, "ema.safetensors"), self.pinned_state_dict)
                else:
                    torch.save(ema.state_dict(), os.path.join(save_dir, "ema.pt"))

            if sampler is not None:
                # only for VariableVideoBatchSampler
                torch.save(sampler.state_dict(step), os.path.join(save_dir, "sampler"))

            if optimizer is not None and include_master_weights:
                self.master_writer = async_save(
                    os.path.join(save_dir, "master.safetensors"), self.master_pinned_state_dict
                )

        dist.barrier()
        return save_dir

    def load(
        self,
        booster: Booster,
        load_dir: str,
        model: nn.Module = None,
        ema: nn.Module = None,
        optimizer: Optimizer = None,
        lr_scheduler: _LRScheduler = None,
        sampler=None,
        strict: bool = False,
        include_master_weights: bool = False,
    ) -> tuple[int, int]:
        """
        Load a checkpoint.

        Args:
            booster (Booster): The Booster object.
            load_dir (str): The directory to load the checkpoint from.
            model (nn.Module): The model to load the checkpoint into.
            ema (nn.Module): The EMA model to load the checkpoint into.
            optimizer (Optimizer): The optimizer to load the checkpoint into.
            lr_scheduler (_LRScheduler): The learning rate scheduler to load the checkpoint into.
            sampler: The sampler to load the checkpoint into.

        Returns:
            tuple[int, int]: The epoch and step of the checkpoint.
        """
        assert os.path.exists(load_dir), f"Checkpoint directory {load_dir} does not exist"
        assert os.path.exists(os.path.join(load_dir, "running_states.json")), "running_states.json does not exist"

        running_states = load_json(os.path.join(load_dir, "running_states.json"))
        if model is not None:
            booster.load_model(
                model,
                _search_valid_path(os.path.join(load_dir, "model")),
                strict=strict,
                low_cpu_mem_mode=False,
                num_threads=32,
            )
        if ema is not None:
            if os.path.exists(os.path.join(load_dir, "ema.safetensors")):
                ema_state_dict = load_file(os.path.join(load_dir, "ema.safetensors"))
            else:
                ema_state_dict = torch.load(os.path.join(load_dir, "ema.pt"), map_location=torch.device("cpu"))
            # ema is not boosted, so we don't use booster.load_model
            ema.load_state_dict(ema_state_dict, strict=strict, assign=True)

        if optimizer is not None:
            booster.load_optimizer(
                optimizer, os.path.join(load_dir, "optimizer"), low_cpu_mem_mode=False, num_threads=32
            )
            if include_master_weights:
                master_state_dict = load_file(os.path.join(load_dir, "master.safetensors"))
                load_master_weights(model, optimizer, master_state_dict)
        if lr_scheduler is not None:
            booster.load_lr_scheduler(lr_scheduler, os.path.join(load_dir, "lr_scheduler"))
        if sampler is not None:
            sampler.load_state_dict(torch.load(os.path.join(load_dir, "sampler")))

        dist.barrier()

        return (running_states["epoch"], running_states["step"])

### シード固定

In [ ]:
from colossalai.utils import set_seed
set_seed(cfg.get("seed", 1024))

### ピンメモリー設定

In [ ]:
import threading
from typing import Dict, List, Optional
import torch

class PinMemoryCache:
    force_dtype: Optional[torch.dtype] = None
    min_cache_numel: int = 0
    pre_alloc_numels: List[int] = []

    def __init__(self):
        self.cache: Dict[int, torch.Tensor] = {}
        self.output_to_cache: Dict[int, int] = {}
        self.cache_to_output: Dict[int, int] = {}
        self.lock = threading.Lock()
        self.total_cnt = 0
        self.hit_cnt = 0

        if len(self.pre_alloc_numels) > 0 and self.force_dtype is not None:
            for n in self.pre_alloc_numels:
                cache_tensor = torch.empty(n, dtype=self.force_dtype, device="cpu", pin_memory=True)
                with self.lock:
                    self.cache[id(cache_tensor)] = cache_tensor

    def get(self, tensor: torch.Tensor) -> torch.Tensor:
        """Receive a cpu tensor and return the corresponding pinned tensor. Note that this only manage memory allocation, doesn't copy content.

        Args:
            tensor (torch.Tensor): The tensor to be pinned.

        Returns:
            torch.Tensor: The pinned tensor.
        """
        self.total_cnt += 1
        with self.lock:
            # find free cache
            for cache_id, cache_tensor in self.cache.items():
                if cache_id not in self.cache_to_output and cache_tensor.numel() >= tensor.numel():
                    target_cache_tensor = cache_tensor[: tensor.numel()].view(tensor.shape)
                    out_id = id(target_cache_tensor)
                    self.output_to_cache[out_id] = cache_id
                    self.cache_to_output[cache_id] = out_id
                    self.hit_cnt += 1
                    return target_cache_tensor
        # no free cache, create a new one
        dtype = self.force_dtype if self.force_dtype is not None else tensor.dtype
        cache_numel = max(tensor.numel(), self.min_cache_numel)
        cache_tensor = torch.empty(cache_numel, dtype=dtype, device="cpu", pin_memory=True)
        target_cache_tensor = cache_tensor[: tensor.numel()].view(tensor.shape)
        out_id = id(target_cache_tensor)
        with self.lock:
            self.cache[id(cache_tensor)] = cache_tensor
            self.output_to_cache[out_id] = id(cache_tensor)
            self.cache_to_output[id(cache_tensor)] = out_id
        return target_cache_tensor

    def remove(self, output_tensor: torch.Tensor) -> None:
        """Release corresponding cache tensor.

        Args:
            output_tensor (torch.Tensor): The tensor to be released.
        """
        out_id = id(output_tensor)
        with self.lock:
            if out_id not in self.output_to_cache:
                raise ValueError("Tensor not found in cache.")
            cache_id = self.output_to_cache.pop(out_id)
            del self.cache_to_output[cache_id]

    def __str__(self):
        with self.lock:
            num_cached = len(self.cache)
            num_used = len(self.output_to_cache)
            total_cache_size = sum([v.numel() * v.element_size() for v in self.cache.values()])
        return f"PinMemoryCache(num_cached={num_cached}, num_used={num_used}, total_cache_size={total_cache_size / 1024**3:.2f} GB, hit rate={self.hit_cnt / self.total_cnt:.2f})"

PinMemoryCache.force_dtype = dtype
pin_memory_cache_pre_alloc_numels = cfg.get("pin_memory_cache_pre_alloc_numels", None)
PinMemoryCache.pre_alloc_numels = pin_memory_cache_pre_alloc_numels

### CollosalAI Boosterの初期化

In [ ]:
# == init ColossalAI booster ==
plugin_type = cfg.get("plugin", "zero2")
plugin_config = cfg.get("plugin_config", {})

In [ ]:
_GLOBAL_PARALLEL_GROUPS = dict()

def set_data_parallel_group(group: dist.ProcessGroup):
    _GLOBAL_PARALLEL_GROUPS["data"] = group

In [ ]:
import random
import warnings
from collections import OrderedDict
from datetime import timedelta

import torch
import torch.distributed as dist
import torch.nn.functional as F
from colossalai.booster.plugin import HybridParallelPlugin, LowLevelZeroPlugin
from colossalai.cluster import DistCoordinator
from colossalai.utils import get_current_device
from einops import rearrange
from torch import nn
from torch.optim.lr_scheduler import _LRScheduler
from tqdm import tqdm

def create_colossalai_plugin(
    plugin: str,
    dtype: str,
    grad_clip: float,
    **kwargs,
) -> LowLevelZeroPlugin | HybridParallelPlugin:
    """
    Create a ColossalAI plugin.

    Args:
        plugin (str): The plugin name.
        dtype (str): The data type.
        grad_clip (float): The gradient clip value.

    Returns:
        LowLevelZeroPlugin |  HybridParallelPlugin: The plugin.
    """
    plugin_kwargs = dict(
        precision=dtype,
        initial_scale=2**16,
        max_norm=grad_clip,
        overlap_allgather=True,
        cast_inputs=False,
        reduce_bucket_size_in_m=20,
    )
    plugin_kwargs.update(kwargs)
    sp_size = plugin_kwargs.get("sp_size", 1)
    if plugin == "zero1" or plugin == "zero2":
        assert sp_size == 1, "Zero plugin does not support sequence parallelism"
        stage = 1 if plugin == "zero1" else 2
        plugin = LowLevelZeroPlugin(
            stage=stage,
            **plugin_kwargs,
        )
        set_data_parallel_group(dist.group.WORLD)
    elif plugin == "hybrid":
        plugin_kwargs["find_unused_parameters"] = True
        reduce_bucket_size_in_m = plugin_kwargs.pop("reduce_bucket_size_in_m")
        if "zero_bucket_size_in_m" not in plugin_kwargs:
            plugin_kwargs["zero_bucket_size_in_m"] = reduce_bucket_size_in_m
        plugin_kwargs.pop("cast_inputs")
        plugin_kwargs["enable_metadata_cache"] = False

        custom_policy = plugin_kwargs.pop("custom_policy", None)
        if custom_policy is not None:
            custom_policy = custom_policy()
        plugin = HybridParallelPlugin(
            custom_policy=custom_policy,
            **plugin_kwargs,
        )
        set_tensor_parallel_group(plugin.tp_group)
        set_sequence_parallel_group(plugin.sp_group)
        set_data_parallel_group(plugin.dp_group)
    else:
        raise ValueError(f"Unknown plugin {plugin}")
    return plugin

In [ ]:
plugin = (
    create_colossalai_plugin(
        plugin=plugin_type,
        dtype=cfg.get("dtype", "bf16"),
        grad_clip=cfg.get("grad_clip", 0),
        **plugin_config,
    )
    if plugin_type != "none"
    else None
)

In [ ]:
booster = Booster(plugin=plugin)

### 実験用ディレクトリの作成

In [ ]:
import argparse
import ast
import json
import os
from datetime import datetime
import torch
from mmengine.config import Config

In [ ]:
def is_distributed() -> bool:
    """
    Check if the code is running in a distributed setting.

    Returns:
        bool: True if running in a distributed setting, False otherwise
    """
    return os.environ.get("WORLD_SIZE", None) is not None

In [ ]:
def is_main_process() -> bool:
    """
    Check if the current process is the main process.

    Returns:
        bool: True if the current process is the main process, False otherwise.
    """
    return not is_distributed() or dist.get_rank() == 0

In [ ]:
def config_to_name(cfg: Config) -> str:
    filename = cfg._filename
    filename = filename.replace("configs/", "")
    filename = filename.replace(".py", "")
    filename = filename.replace("/", "_")
    return filename

In [ ]:
def sync_string(value: str):
    """
    This function synchronizes a string across all processes.
    """
    if not is_distributed():
        return value
    bytes_value = value.encode("utf-8")
    max_len = 256
    bytes_tensor = torch.zeros(max_len, dtype=torch.uint8).cuda()
    bytes_tensor[: len(bytes_value)] = torch.tensor(
        list(bytes_value), dtype=torch.uint8
    )
    torch.distributed.broadcast(bytes_tensor, 0)
    synced_value = bytes_tensor.cpu().numpy().tobytes().decode("utf-8").rstrip("\x00")
    return synced_value

In [ ]:
def create_experiment_workspace(
    output_dir: str, model_name: str = None, config: dict = None, exp_name: str = None
) -> tuple[str, str]:
    """
    This function creates a folder for experiment tracking.

    Args:
        output_dir: The path to the output directory.
        model_name: The name of the model.
        exp_name: The given name of the experiment, if None will use default.

    Returns:
        tuple[str, str]: The experiment name and the experiment directory.
    """
    if exp_name is None:
        # Make outputs folder (holds all experiment subfolders)
        experiment_index = datetime.now().strftime("%y%m%d_%H%M%S")
        experiment_index = sync_string(experiment_index)
        # Create an experiment folder
        model_name = (
            "-" + model_name.replace("/", "-") if model_name is not None else ""
        )
        exp_name = f"{experiment_index}{model_name}"
    exp_dir = f"{output_dir}/{exp_name}"
    if is_main_process():
        os.makedirs(exp_dir, exist_ok=True)
        # Save the config
        with open(f"{exp_dir}/config.txt", "w", encoding="utf-8") as f:
            json.dump(config, f, indent=4)

    return exp_name, exp_dir

# == init exp_dir ==
exp_name, exp_dir = create_experiment_workspace(
    cfg.get("outputs", "./outputs"),
    model_name=config_to_name(cfg),
    config=cfg.to_dict(),
)

In [ ]:
def is_pipeline_enabled(plugin_type: str, plugin_config: dict) -> bool:
    return plugin_type == "hybrid" and plugin_config.get("pp_size", 1) > 1

In [ ]:
def is_log_process(plugin_type: str, plugin_config: dict) -> bool:
    if is_pipeline_enabled(plugin_type, plugin_config):
        return dist.get_rank() == dist.get_world_size() - 1
    return dist.get_rank() == 0

In [ ]:
if is_log_process(plugin_type, plugin_config):
    print(f"changing {exp_dir} to share")
    os.system(f"chgrp -R share {exp_dir}")

### ロガーの初期化

In [ ]:
def create_logger(logging_dir: str = None) -> logging.Logger:
    """
    Create a logger that writes to a log file and stdout. Only the main process logs.

    Args:
        logging_dir (str): The directory to save the log file.

    Returns:
        logging.Logger: The logger.
    """
    if is_main_process():
        additional_args = dict()
        if logging_dir is not None:
            additional_args["handlers"] = [
                logging.StreamHandler(),
                logging.FileHandler(f"{logging_dir}/log.txt"),
            ]
        logging.basicConfig(
            level=logging.INFO,
            format="[\033[34m%(asctime)s\033[0m] %(message)s",
            datefmt="%Y-%m-%d %H:%M:%S",
            **additional_args,
        )
        logger = logging.getLogger(__name__)
        if logging_dir is not None:
            logger.info("Experiment directory created at %s", logging_dir)
    else:
        logger = logging.getLogger(__name__)
        logger.addHandler(logging.NullHandler())
    return logger

from pprint import pformat

logger = create_logger(exp_dir)
logger.info("Training configuration:\n %s", pformat(cfg.to_dict()))

### TensorboardとWandBの初期化

In [ ]:
from torch.utils.tensorboard import SummaryWriter

In [ ]:
def create_tensorboard_writer(exp_dir: str) -> SummaryWriter:
    """
    Create a tensorboard writer.

    Args:
        exp_dir (str): The directory to save tensorboard logs.

    Returns:
        SummaryWriter: The tensorboard writer.
    """
    tensorboard_dir = f"{exp_dir}/tensorboard"
    os.makedirs(tensorboard_dir, exist_ok=True)
    writer = SummaryWriter(tensorboard_dir)
    return writer

In [ ]:
# == init logger, tensorboard & wandb ==
tb_writer = None

# True
if coordinator.is_master():
    tb_writer = create_tensorboard_writer(exp_dir)

    # False
    if cfg.get("wandb", False):
        wandb.init(
            project=cfg.get("wandb_project", "Open-Sora"),
            name=cfg.get("wandb_expr_name", exp_name),
            config=cfg.to_dict(),
            dir=exp_dir,
        )

### データセットの構築

In [ ]:
# ======================================================
# 2. build dataset and dataloader
# ======================================================
logger.info(f"Building dataset... {cfg.dataset=}")

In [ ]:
cfg.dataset["data_path"] = PEXELS_DATASET_DIR
cfg.dataset

In [ ]:
VALID_KEYS = ("neg", "path")

In [ ]:
# @DATASETS.register_module("text")
class TextDataset(torch.utils.data.Dataset):
    """
    Dataset for text data
    """

    def __init__(
        self,
        data_path: str = None,
        tokenize_fn: callable = None,
        fps_max: int = 16,
        vmaf: bool = False,
        memory_efficient: bool = False,
        **kwargs,
    ):
        self.data_path = data_path
        self.data = read_file(data_path, memory_efficient=memory_efficient)
        self.memory_efficient = memory_efficient
        self.tokenize_fn = tokenize_fn
        self.vmaf = vmaf

        if fps_max is not None:
            self.fps_max = fps_max
        else:
            self.fps_max = 999999999

    def to_efficient(self):
        if self.memory_efficient:
            addition_data_path = self.data_path.split(".")[0]
            self._data = self.data
            self.data = EfficientParquet(self._data, addition_data_path)

    def getitem(self, index: int) -> dict:
        ret = dict()
        sample = self.data.iloc[index].to_dict()
        sample_fps = sample.get("fps", np.nan)
        new_fps, sampling_interval = map_target_fps(sample_fps, self.fps_max)
        ret.update({"sampling_interval": sampling_interval})

        if "text" in sample:
            ret["text"] = sample.pop("text")
            postfixs = []
            if new_fps != 0 and self.fps_max < 999:
                postfixs.append(f"{new_fps} FPS")
            if self.vmaf and "score_vmafmotion" in sample and not np.isnan(sample["score_vmafmotion"]):
                postfixs.append(f"{int(sample['score_vmafmotion'] + 0.5)} motion score")
            postfix = " " + ", ".join(postfixs) + "." if postfixs else ""
            ret["text"] = ret["text"] + postfix
            if self.tokenize_fn is not None:
                ret.update({k: v.squeeze(0) for k, v in self.tokenize_fn(ret["text"]).items()})

        if "ref" in sample:  # i2v & v2v reference
            ret["ref"] = sample.pop("ref")

        # name of the generated sample
        if "name" in sample:  # sample name (`dataset_idx`)
            ret["name"] = sample.pop("name")
        else:
            ret["index"] = index  # use index for name
        valid_sample = {k: v for k, v in sample.items() if k in VALID_KEYS}
        ret.update(valid_sample)
        return ret

    def __getitem__(self, index):
        return self.getitem(index)

    def __len__(self):
        return len(self.data)

In [ ]:

def read_file(input_path, memory_efficient=False):
    if input_path.endswith(".csv"):
        assert not memory_efficient, "Memory efficient mode is not supported for CSV files"
        return pd.read_csv(input_path)
    elif input_path.endswith(".parquet"):
        columns = None
        if memory_efficient:
            columns = ["path", "num_frames", "height", "width", "aspect_ratio", "fps", "resolution"]
        if SUPPORT_DASK:
            ret = dd.read_parquet(input_path, columns=columns).compute()
        else:
            ret = pd.read_parquet(input_path, columns=columns)
        return ret
    else:
        raise NotImplementedError(f"Unsupported file format: {input_path}")

In [ ]:
def is_img(path):
    ext = os.path.splitext(path)[-1].lower()
    return ext in IMG_EXTENSIONS

In [ ]:
import gc
import math
import os
import re
import warnings
from fractions import Fraction

import av
import cv2
import numpy as np
import torch
from torchvision import get_video_backend
from torchvision.io.video import _check_av_available


In [ ]:
def _read_from_stream(
    video_frames,
    container: "av.container.Container",
    start_offset: float,
    end_offset: float,
    pts_unit: str,
    stream: "av.stream.Stream",
    stream_name: dict[str, int | tuple[int, ...] | list[int] | None],
    filename: str | None = None,
) -> list["av.frame.Frame"]:
    if pts_unit == "sec":
        # TODO: we should change all of this from ground up to simply take
        # sec and convert to MS in C++
        start_offset = int(math.floor(start_offset * (1 / stream.time_base)))
        if end_offset != float("inf"):
            end_offset = int(math.ceil(end_offset * (1 / stream.time_base)))
    else:
        warnings.warn("The pts_unit 'pts' gives wrong results. Please use pts_unit 'sec'.")

    should_buffer = True
    max_buffer_size = 5
    if stream.type == "video":
        # DivX-style packed B-frames can have out-of-order pts (2 frames in a single pkt)
        # so need to buffer some extra frames to sort everything
        # properly
        extradata = stream.codec_context.extradata
        # overly complicated way of finding if `divx_packed` is set, following
        # https://github.com/FFmpeg/FFmpeg/commit/d5a21172283572af587b3d939eba0091484d3263
        if extradata and b"DivX" in extradata:
            # can't use regex directly because of some weird characters sometimes...
            pos = extradata.find(b"DivX")
            d = extradata[pos:]
            o = re.search(rb"DivX(\d+)Build(\d+)(\w)", d)
            if o is None:
                o = re.search(rb"DivX(\d+)b(\d+)(\w)", d)
            if o is not None:
                should_buffer = o.group(3) == b"p"
    seek_offset = start_offset
    # some files don't seek to the right location, so better be safe here
    seek_offset = max(seek_offset - 1, 0)
    if should_buffer:
        # FIXME this is kind of a hack, but we will jump to the previous keyframe
        # so this will be safe
        seek_offset = max(seek_offset - max_buffer_size, 0)
    try:
        # TODO check if stream needs to always be the video stream here or not
        container.seek(seek_offset, any_frame=False, backward=True, stream=stream)
    except av.AVError as e:
        print(f"[Warning] Error while seeking video {filename}: {e}")
        return []

    # == main ==
    buffer_count = 0
    frames_pts = []
    cnt = 0
    try:
        for _idx, frame in enumerate(container.decode(**stream_name)):
            frames_pts.append(frame.pts)
            video_frames[cnt] = frame.to_rgb().to_ndarray()
            cnt += 1
            if cnt >= len(video_frames):
                break
            if frame.pts >= end_offset:
                if should_buffer and buffer_count < max_buffer_size:
                    buffer_count += 1
                    continue
                break
    except av.AVError as e:
        print(f"[Warning] Error while reading video {filename}: {e}")

    # garbage collection for thread leakage
    container.close()
    del container
    # NOTE: manually garbage collect to close pyav threads
    gc.collect()

    # ensure that the results are sorted wrt the pts
    # NOTE: here we assert frames_pts is sorted
    start_ptr = 0
    end_ptr = cnt
    while start_ptr < end_ptr and frames_pts[start_ptr] < start_offset:
        start_ptr += 1
    while start_ptr < end_ptr and frames_pts[end_ptr - 1] > end_offset:
        end_ptr -= 1
    if start_offset > 0 and start_offset not in frames_pts[start_ptr:end_ptr]:
        # if there is no frame that exactly matches the pts of start_offset
        # add the last frame smaller than start_offset, to guarantee that
        # we will have all the necessary data. This is most useful for audio
        if start_ptr > 0:
            start_ptr -= 1
    result = video_frames[start_ptr:end_ptr].copy()
    return result

In [ ]:
def read_video_av(
    filename: str,
    start_pts: float | Fraction = 0,
    end_pts: float | Fraction | None = None,
    pts_unit: str = "pts",
    output_format: str = "THWC",
) -> tuple[torch.Tensor, torch.Tensor, dict]:
    """
    Reads a video from a file, returning both the video frames and the audio frames

    This method is modified from torchvision.io.video.read_video, with the following changes:

    1. will not extract audio frames and return empty for aframes
    2. remove checks and only support pyav
    3. add container.close() and gc.collect() to avoid thread leakage
    4. try our best to avoid memory leak

    Args:
        filename (str): path to the video file
        start_pts (int if pts_unit = 'pts', float / Fraction if pts_unit = 'sec', optional):
            The start presentation time of the video
        end_pts (int if pts_unit = 'pts', float / Fraction if pts_unit = 'sec', optional):
            The end presentation time
        pts_unit (str, optional): unit in which start_pts and end_pts values will be interpreted,
            either 'pts' or 'sec'. Defaults to 'pts'.
        output_format (str, optional): The format of the output video tensors. Can be either "THWC" (default) or "TCHW".

    Returns:
        vframes (Tensor[T, H, W, C] or Tensor[T, C, H, W]): the `T` video frames
        aframes (Tensor[K, L]): the audio frames, where `K` is the number of channels and `L` is the number of points
        info (dict): metadata for the video and audio. Can contain the fields video_fps (float) and audio_fps (int)
    """
    # format
    output_format = output_format.upper()
    if output_format not in ("THWC", "TCHW"):
        raise ValueError(f"output_format should be either 'THWC' or 'TCHW', got {output_format}.")
    # file existence
    if not os.path.exists(filename):
        raise RuntimeError(f"File not found: {filename}")
    # backend check
    assert get_video_backend() == "pyav", "pyav backend is required for read_video_av"
    _check_av_available()
    # end_pts check
    if end_pts is None:
        end_pts = float("inf")
    if end_pts < start_pts:
        raise ValueError(f"end_pts should be larger than start_pts, got start_pts={start_pts} and end_pts={end_pts}")

    # == get video info ==
    info = {}
    # TODO: creating an container leads to memory leak (1G for 8 workers 1 GPU)
    container = av.open(filename, metadata_errors="ignore")
    # fps
    video_fps = container.streams.video[0].average_rate
    # guard against potentially corrupted files
    if video_fps is not None:
        info["video_fps"] = float(video_fps)
    iter_video = container.decode(**{"video": 0})
    frame = next(iter_video).to_rgb().to_ndarray()
    height, width = frame.shape[:2]
    total_frames = container.streams.video[0].frames
    if total_frames == 0:
        total_frames = MAX_NUM_FRAMES
        warnings.warn(f"total_frames is 0, using {MAX_NUM_FRAMES} as a fallback")
    container.close()
    del container

    # HACK: must create before iterating stream
    # use np.zeros will not actually allocate memory
    # use np.ones will lead to a little memory leak
    video_frames = np.zeros((total_frames, height, width, 3), dtype=np.uint8)

    # == read ==
    try:
        # TODO: The reading has memory leak (4G for 8 workers 1 GPU)
        container = av.open(filename, metadata_errors="ignore")
        assert container.streams.video is not None
        video_frames = _read_from_stream(
            video_frames,
            container,
            start_pts,
            end_pts,
            pts_unit,
            container.streams.video[0],
            {"video": 0},
            filename=filename,
        )
    except av.AVError as e:
        print(f"[Warning] Error while reading video {filename}: {e}")

    vframes = torch.from_numpy(video_frames).clone()
    del video_frames
    if output_format == "TCHW":
        # [T,H,W,C] --> [T,C,H,W]
        vframes = vframes.permute(0, 3, 1, 2)

    aframes = torch.empty((1, 0), dtype=torch.float32)
    return vframes, aframes, info

In [ ]:
def read_video(video_path, backend="av"):
    if backend == "cv2":
        vframes, vinfo = read_video_cv2(video_path)
    elif backend == "av":
        vframes, _, vinfo = read_video_av(filename=video_path, pts_unit="sec", output_format="TCHW")
    else:
        raise ValueError

    return vframes, vinfo


In [ ]:
class TemporalRandomCrop(object):
    """Temporally crop the given frame indices at a random location.

    Args:
            size (int): Desired length of frames will be seen in the model.
    """

    def __init__(self, size):
        self.size = size

    def __call__(self, total_frames):
        rand_end = max(0, total_frames - self.size - 1)
        begin_index = random.randint(0, rand_end)
        end_index = min(begin_index + self.size, total_frames)
        return begin_index, end_index

In [ ]:
def temporal_random_crop(
    vframes: torch.Tensor, num_frames: int, frame_interval: int, return_frame_indices: bool = False
) -> torch.Tensor | tuple[torch.Tensor, np.ndarray]:
    # temporal_sample = video_transforms.temporalrandomcrop(num_frames * frame_interval)
    temporal_sample = TemporalRandomCrop(num_frames * frame_interval)

    total_frames = len(vframes)
    start_frame_ind, end_frame_ind = temporal_sample(total_frames)

    assert (
        end_frame_ind - start_frame_ind >= num_frames
    ), f"Not enough frames to sample, {end_frame_ind} - {start_frame_ind} < {num_frames}"

    frame_indices = np.linspace(start_frame_ind, end_frame_ind - 1, num_frames, dtype=int)
    video = vframes[frame_indices]
    if return_frame_indices:
        return video, frame_indices
    else:
        return video

In [ ]:
def _is_tensor_video_clip(clip):
    if not torch.is_tensor(clip):
        raise TypeError("clip should be Tensor. Got %s" % type(clip))

    if not clip.ndimension() == 4:
        raise ValueError("clip should be 4D. Got %dD" % clip.dim())

    return True

In [ ]:
def to_tensor(clip):
    """
    Convert tensor data type from uint8 to float, divide value by 255.0 and
    permute the dimensions of clip tensor
    Args:
        clip (torch.tensor, dtype=torch.uint8): Size is (T, C, H, W)
    Return:
        clip (torch.tensor, dtype=torch.float): Size is (T, C, H, W)
    """
    _is_tensor_video_clip(clip)
    if not clip.dtype == torch.uint8:
        raise TypeError("clip tensor should have data type uint8. Got %s" % str(clip.dtype))
    # return clip.float().permute(3, 0, 1, 2) / 255.0
    return clip.float() / 255.0

In [ ]:
class ToTensorVideo:
    """
    Convert tensor data type from uint8 to float, divide value by 255.0 and
    permute the dimensions of clip tensor
    """

    def __init__(self):
        pass

    def __call__(self, clip):
        """
        Args:
            clip (torch.tensor, dtype=torch.uint8): Size is (T, C, H, W)
        Return:
            clip (torch.tensor, dtype=torch.float): Size is (T, C, H, W)
        """
        return to_tensor(clip)

    def __repr__(self) -> str:
        return self.__class__.__name__

In [ ]:

class UCFCenterCropVideo:
    """
    First scale to the specified size in equal proportion to the short edge,
    then center cropping
    """

    def __init__(
        self,
        size,
        interpolation_mode="bilinear",
    ):
        if isinstance(size, tuple):
            if len(size) != 2:
                raise ValueError(f"size should be tuple (height, width), instead got {size}")
            self.size = size
        else:
            self.size = (size, size)

        self.interpolation_mode = interpolation_mode

    def __call__(self, clip):
        """
        Args:
            clip (torch.tensor): Video clip to be cropped. Size is (T, C, H, W)
        Returns:
            torch.tensor: scale resized / center cropped video clip.
                size is (T, C, crop_size, crop_size)
        """
        clip_resize = resize_scale(clip=clip, target_size=self.size, interpolation_mode=self.interpolation_mode)
        clip_center_crop = center_crop(clip_resize, self.size)
        return clip_center_crop

    def __repr__(self) -> str:
        return f"{self.__class__.__name__}(size={self.size}, interpolation_mode={self.interpolation_mode}"

In [ ]:
import numbers

# - This file is adapted from https://github.com/Vchitect/Latte/blob/main/datasets/video_transforms.py
import random

import numpy as np
import torch

In [ ]:
def resize(clip, target_size, interpolation_mode):
    if len(target_size) != 2:
        raise ValueError(f"target size should be tuple (height, width), instead got {target_size}")
    return torch.nn.functional.interpolate(clip, size=target_size, mode=interpolation_mode, align_corners=False)


In [ ]:
def crop(clip, i, j, h, w):
    """
    Args:
        clip (torch.tensor): Video clip to be cropped. Size is (T, C, H, W)
    """
    if len(clip.size()) != 4:
        raise ValueError("clip should be a 4D tensor")
    return clip[..., i : i + h, j : j + w]

In [ ]:
def resize_crop_to_fill(clip, target_size):
    if not _is_tensor_video_clip(clip):
        raise ValueError("clip should be a 4D torch.tensor")
    h, w = clip.size(-2), clip.size(-1)
    th, tw = target_size[0], target_size[1]
    rh, rw = th / h, tw / w
    if rh > rw:
        sh, sw = th, round(w * rh)
        clip = resize(clip, (sh, sw), "bilinear")
        i = 0
        j = int(round(sw - tw) / 2.0)
    else:
        sh, sw = round(h * rw), tw
        clip = resize(clip, (sh, sw), "bilinear")
        i = int(round(sh - th) / 2.0)
        j = 0
    assert i + th <= clip.size(-2) and j + tw <= clip.size(-1)
    return crop(clip, i, j, th, tw)

In [ ]:
class ResizeCrop:
    def __init__(self, size):
        if isinstance(size, numbers.Number):
            self.size = (int(size), int(size))
        else:
            self.size = size

    def __call__(self, clip):
        clip = resize_crop_to_fill(clip, self.size)
        return clip

    def __repr__(self) -> str:
        return f"{self.__class__.__name__}(size={self.size})"


In [ ]:
def get_transforms_video(name="center", image_size=(256, 256)):
    if name is None:
        return None
    elif name == "center":
        assert image_size[0] == image_size[1], "image_size must be square for center crop"
        transform_video = transforms.Compose(
            [
                # video_transforms.ToTensorVideo(),  # TCHW
                ToTensorVideo(),  # TCHW
                # video_transforms.RandomHorizontalFlipVideo(),
                UCFCenterCropVideo(image_size[0]),
                # video_transforms.UCFCenterCropVideo(image_size[0]),
                transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5], inplace=True),
            ]
        )
    elif name == "resize_crop":
        transform_video = transforms.Compose(
            [
                # video_transforms.ToTensorVideo(),  # TCHW
                ToTensorVideo(),  # TCHW
                # video_transforms.ResizeCrop(image_size),
                ResizeCrop(image_size),
                transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5], inplace=True),
            ]
        )
    elif name == "rand_size_crop":
        transform_video = transforms.Compose(
            [
                # video_transforms.ToTensorVideo(),  # TCHW
                ToTensorVideo(),  # TCHW
                video_transforms.RandomSizedCrop(image_size),
                transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5], inplace=True),
            ]
        )
    else:
        raise NotImplementedError(f"Transform {name} not implemented")
    return transform_video

In [ ]:
# @DATASETS.register_module("video_text")
class VideoTextDataset(TextDataset):
    def __init__(
        self,
        transform_name: str = None,
        bucket_class: str = "Bucket",
        rand_sample_interval: int = None,  # random sample_interval value from [1, min(rand_sample_interval, video_allowed_max)]
        **kwargs,
    ):
        super().__init__(**kwargs)
        self.transform_name = transform_name
        self.bucket_class = bucket_class
        self.rand_sample_interval = rand_sample_interval

    def get_image(self, index: int, height: int, width: int) -> dict:
        sample = self.data.iloc[index]
        path = sample["path"]
        # loading
        image = pil_loader(path)

        # transform
        transform = get_transforms_image(self.transform_name, (height, width))
        image = transform(image)

        # CHW -> CTHW
        video = image.unsqueeze(1)

        return {"video": video}

    def get_video(self, index: int, num_frames: int, height: int, width: int, sampling_interval: int) -> dict:
        sample = self.data.iloc[index]
        path = sample["path"]

        # loading
        vframes, vinfo = read_video(path, backend="av")

        if self.rand_sample_interval is not None:
            # randomly sample from 1 - self.rand_sample_interval
            video_allowed_max = min(len(vframes) // num_frames, self.rand_sample_interval)
            sampling_interval = random.randint(1, video_allowed_max)

        # Sampling video frames
        video = temporal_random_crop(vframes, num_frames, sampling_interval)

        video = video.clone()
        del vframes

        # transform
        transform = get_transforms_video(self.transform_name, (height, width))
        video = transform(video)  # T C H W
        video = video.permute(1, 0, 2, 3)

        ret = {"video": video}

        return ret

    def get_image_or_video(self, index: int, num_frames: int, height: int, width: int, sampling_interval: int) -> dict:
        sample = self.data.iloc[index]
        path = sample["path"]

        if is_img(path):
            return self.get_image(index, height, width)
        return self.get_video(index, num_frames, height, width, sampling_interval)

    def getitem(self, index: str) -> dict:
        # a hack to pass in the (time, height, width) info from sampler
        index, num_frames, height, width = [int(val) for val in index.split("-")]
        ret = dict()
        ret.update(super().getitem(index))
        try:
            ret.update(self.get_image_or_video(index, num_frames, height, width, ret["sampling_interval"]))
        except Exception as e:
            path = self.data.iloc[index]["path"]
            print(f"video {path}: {e}")
            return None
        return ret

    def __getitem__(self, index):
        return self.getitem(index)

# cfg.dataset =
# {'type': 'video_text',
#  'transform_name': 'resize_crop',
#  'data_path': '/workspaces/open-sora/Open-Sora/datasets/pexels_5',
#  'fps_max': 24}

dataset = VideoTextDataset(
    type = "video_text",
    transform_name = "resize_crop",
    data_path = PEXELS_CSV_PATH,
    fps_max = 24,
)
logger.info("Dataset contains %s samples.", len(dataset))

### データローダの構築

In [ ]:
cache_pin_memory = pin_memory_cache_pre_alloc_numels is not None
cache_pin_memory

In [ ]:
def get_data_parallel_group(get_mixed_dp_pg : bool = False):
    if get_mixed_dp_pg and "mixed_dp_group" in _GLOBAL_PARALLEL_GROUPS:
        return _GLOBAL_PARALLEL_GROUPS["mixed_dp_group"]
    return _GLOBAL_PARALLEL_GROUPS.get("data", dist.group.WORLD)

In [ ]:
cfg["num_workers"] = 0
cfg["prefetch_factor"] = None

In [ ]:
dataloader_args = dict(
    dataset=dataset,
    batch_size=cfg.get("batch_size", None),
    num_workers=cfg.get("num_workers", 4),
    seed=cfg.get("seed", 1024),
    shuffle=True,
    drop_last=True,
    pin_memory=True,
    process_group=get_data_parallel_group(),
    prefetch_factor=cfg.get("prefetch_factor", None),
    cache_pin_memory=cache_pin_memory,
)

dataloader_args

In [ ]:
import collections
import functools
import os
import queue
import random
import threading

import numpy as np
import torch
import torch.multiprocessing as multiprocessing
from torch._utils import ExceptionWrapper
from torch.distributed import ProcessGroup
from torch.utils.data import DataLoader, _utils
from torch.utils.data._utils import MP_STATUS_CHECK_INTERVAL
from torch.utils.data.dataloader import (
    IterDataPipe,
    MapDataPipe,
    _BaseDataLoaderIter,
    _MultiProcessingDataLoaderIter,
    _sharding_worker_init_fn,
    _SingleProcessDataLoaderIter,
)


In [ ]:
from collections import OrderedDict, defaultdict
from typing import Iterator

import numpy as np
import torch
import torch.distributed as dist
from torch.utils.data import Dataset, DistributedSampler
from collections import OrderedDict
import numpy as np


In [ ]:
ASPECT_RATIO_LD_LIST = [  # width:height
    "2.39:1",  # cinemascope, 2.39
    "2:1",  # rare, 2
    "16:9",  # rare, 1.89
    "1.85:1",  # american widescreen, 1.85
    "9:16",  # popular, 1.78
    "5:8",  # rare, 1.6
    "3:2",  # rare, 1.5
    "4:3",  # classic, 1.33
    "1:1",  # square
]

In [ ]:

def get_aspect_ratios_dict(
    total_pixels: int = 256 * 256, training: bool = True
) -> dict[str, tuple[int, int]]:
    D = int(os.environ.get("AE_SPATIAL_COMPRESSION", 16))
    aspect_ratios_dict = {}
    aspect_ratios_vertical_dict = {}
    for ratio in ASPECT_RATIO_LD_LIST:
        width_ratio, height_ratio = map(float, ratio.split(":"))
        width = int(math.sqrt(total_pixels * (width_ratio / height_ratio)) // D) * D
        height = int((total_pixels / width) // D) * D

        if training:
            # adjust aspect ratio to match total pixels
            diff = abs(height * width - total_pixels)
            candidate = [
                (height - D, width),
                (height + D, width),
                (height, width - D),
                (height, width + D),
            ]
            for h, w in candidate:
                if abs(h * w - total_pixels) < diff:
                    height, width = h, w
                    diff = abs(h * w - total_pixels)

        # remove duplicated aspect ratio
        if (height, width) not in aspect_ratios_dict.values() or not training:
            aspect_ratios_dict[ratio] = (height, width)
            vertial_ratios = ":".join(ratio.split(":")[::-1])
            aspect_ratios_vertical_dict[vertial_ratios] = (width, height)

    aspect_ratios_dict.update(aspect_ratios_vertical_dict)

    return aspect_ratios_dict

In [ ]:
def get_num_pexels_from_name(resolution: str) -> int:
    resolution = resolution.split("_")[0]
    if resolution.endswith("px"):
        size = int(resolution[:-2])
        num_pexels = size * size
    elif resolution.endswith("p"):
        size = int(resolution[:-1])
        num_pexels = int(size * size / 9 * 16)
    else:
        raise ValueError(f"Invalid resolution {resolution}")
    return num_pexels

In [ ]:
def get_resolution_with_aspect_ratio(
    resolution: str,
) -> tuple[int, dict[str, tuple[int, int]]]:
    """Get resolution with aspect ratio

    Args:
        resolution (str): resolution name. The format is name only or "{name}_{setting}".
            name supports "256px" or "360p". setting supports "ar1:1" or "max".

    Returns:
        tuple[int, dict[str, tuple[int, int]]]: resolution with aspect ratio
    """
    keys = resolution.split("_")
    if len(keys) == 1:
        resolution = keys[0]
        setting = ""
    else:
        resolution, setting = keys
        assert setting == "max" or setting.startswith(
            "ar"
        ), f"Invalid setting {setting}"

    # get resolution
    num_pexels = get_num_pexels_from_name(resolution)

    # get aspect ratio
    aspect_ratio_dict = get_aspect_ratios_dict(num_pexels)

    # handle setting
    if setting == "max":
        aspect_ratio = max(
            aspect_ratio_dict,
            key=lambda x: aspect_ratio_dict[x][0] * aspect_ratio_dict[x][1],
        )
        aspect_ratio_dict = {aspect_ratio: aspect_ratio_dict[aspect_ratio]}
    elif setting.startswith("ar"):
        aspect_ratio = setting[2:]
        assert (
            aspect_ratio in aspect_ratio_dict
        ), f"Aspect ratio {aspect_ratio} not found"
        aspect_ratio_dict = {aspect_ratio: aspect_ratio_dict[aspect_ratio]}

    return num_pexels, aspect_ratio_dict

In [ ]:
def map_target_fps(
    fps: float,
    max_fps: float,
) -> tuple[float, int]:
    """
    Map fps to a new fps that is less than max_fps.

    Args:
        fps (float): Original fps.
        max_fps (float): Maximum fps.

    Returns:
        tuple[float, int]: New fps and sampling interval.
    """
    if math.isnan(fps):
        return 0, 1
    if fps < max_fps:
        return fps, 1
    sampling_interval = math.ceil(fps / max_fps)
    new_fps = math.floor(fps / sampling_interval)
    return new_fps, sampling_interval

In [ ]:
def get_closest_ratio(height: float, width: float, ratios: dict) -> str:
    aspect_ratio = height / width
    closest_ratio = min(
        ratios.keys(), key=lambda ratio: abs(aspect_ratio - get_ratio(ratio))
    )
    return closest_ratio

In [ ]:
def get_ratio(name: str) -> float:
    width, height = map(float, name.split(":"))
    return height / width

In [ ]:
class Bucket:
    def __init__(self, bucket_config: dict[str, dict[int, tuple[float, int] | tuple[tuple[float, float], int]]]):
        """
        Args:
            bucket_config (dict): A dictionary containing the bucket configuration.
                The dictionary should be in the following format:
                {
                    "bucket_name": {
                        "time": (probability, batch_size),
                        "time": (probability, batch_size),
                        ...
                    },
                    ...
                }

                Or in the following format:
                {
                    "bucket_name": {
                        "time": ((probability, next_probability), batch_size),
                        "time": ((probability, next_probability), batch_size),
                        ...
                    },
                    ...
                }
                The bucket_name should be the name of the bucket, and the time should be the number of frames in the video.
                The probability should be a float between 0 and 1, and the batch_size should be an integer.
                If the probability is a tuple, the second value should be the probability to skip to the next time.
        """

        aspect_ratios = {key: get_resolution_with_aspect_ratio(key) for key in bucket_config.keys()}
        bucket_probs = OrderedDict()
        bucket_bs = OrderedDict()
        bucket_names = sorted(bucket_config.keys(), key=lambda x: aspect_ratios[x][0], reverse=True)

        for key in bucket_names:
            bucket_time_names = sorted(bucket_config[key].keys(), key=lambda x: x, reverse=True)
            bucket_probs[key] = OrderedDict({k: bucket_config[key][k][0] for k in bucket_time_names})
            bucket_bs[key] = OrderedDict({k: bucket_config[key][k][1] for k in bucket_time_names})

        self.hw_criteria = {k: aspect_ratios[k][0] for k in bucket_names}
        self.t_criteria = {k1: {k2: k2 for k2 in bucket_config[k1].keys()} for k1 in bucket_names}
        self.ar_criteria = {
            k1: {k2: {k3: v3 for k3, v3 in aspect_ratios[k1][1].items()} for k2 in bucket_config[k1].keys()}
            for k1 in bucket_names
        }

        bucket_id_cnt = num_bucket = 0
        bucket_id = dict()
        for k1, v1 in bucket_probs.items():
            bucket_id[k1] = dict()
            for k2, _ in v1.items():
                bucket_id[k1][k2] = bucket_id_cnt
                bucket_id_cnt += 1
                num_bucket += len(aspect_ratios[k1][1])

        self.bucket_probs = bucket_probs
        self.bucket_bs = bucket_bs
        self.bucket_id = bucket_id
        self.num_bucket = num_bucket

        log_message("Number of buckets: %s", num_bucket)

    def get_bucket_id(
        self,
        T: int,
        H: int,
        W: int,
        fps: float,
        path: str | None = None,
        seed: int | None = None,
        fps_max: int = 16,
    ) -> tuple[str, int, int] | None:
        approx = 0.8
        _, sampling_interval = map_target_fps(fps, fps_max)
        T = T // sampling_interval
        resolution = H * W
        rng = np.random.default_rng(seed)

        # Reference to probabilities and criteria for faster access
        bucket_probs = self.bucket_probs
        hw_criteria = self.hw_criteria
        ar_criteria = self.ar_criteria

        # Start searching for the appropriate bucket
        for hw_id, t_criteria in bucket_probs.items():
            # if resolution is too low, skip
            if resolution < hw_criteria[hw_id] * approx:
                continue

            # if sample is an image
            if T == 1:
                if 1 in t_criteria:
                    if rng.random() < t_criteria[1]:
                        return hw_id, 1, get_closest_ratio(H, W, ar_criteria[hw_id][1])
                continue

            # Look for suitable t_id for video
            for t_id, prob in t_criteria.items():
                if T >= t_id and t_id != 1:
                    # if prob is a tuple, use the second value as the threshold to skip
                    # to the next t_id
                    if isinstance(prob, tuple):
                        next_hw_prob, next_t_prob = prob
                        if next_t_prob >= 1 or rng.random() <= next_t_prob:
                            continue
                    else:
                        next_hw_prob = prob
                    if next_hw_prob >= 1 or rng.random() <= next_hw_prob:
                        ar_id = get_closest_ratio(H, W, ar_criteria[hw_id][t_id])
                        return hw_id, t_id, ar_id
                    else:
                        break

        return None

    def get_thw(self, bucket_idx: tuple[str, int, int]) -> tuple[int, int, int]:
        assert len(bucket_idx) == 3
        T = self.t_criteria[bucket_idx[0]][bucket_idx[1]]
        H, W = self.ar_criteria[bucket_idx[0]][bucket_idx[1]][bucket_idx[2]]
        return T, H, W

    def get_prob(self, bucket_idx: tuple[str, int]) -> float:
        return self.bucket_probs[bucket_idx[0]][bucket_idx[1]]

    def get_batch_size(self, bucket_idx: tuple[str, int]) -> int:
        return self.bucket_bs[bucket_idx[0]][bucket_idx[1]]

    def __len__(self) -> int:
        return self.num_bucket

In [ ]:
def log_message(*args, level: str = "info"):
    """
    Log a message to the logger.

    Args:
        *args: The message to log.
        level (str): The logging level.
    """
    logger = logging.getLogger(__name__)
    if level == "info":
        logger.info(*args)
    elif level == "warning":
        logger.warning(*args)
    elif level == "error":
        logger.error(*args)
    elif level == "print":
        print(*args)
    else:
        raise ValueError(f"Invalid logging level: {level}")

In [ ]:
from pandarallel import pandarallel

In [ ]:
# use pandarallel to accelerate bucket processing
# NOTE: pandarallel should only access local variables
def apply(data, method=None, seed=None, num_bucket=None, fps_max=16):
    return method(
        data["num_frames"],
        data["height"],
        data["width"],
        data["fps"],
        data["path"],
        seed + data["id"] * num_bucket,
        fps_max,
    )

In [ ]:
def sync_object_across_devices(obj: Any, rank: int = 0):
    """
    Synchronizes any picklable object across devices in a PyTorch distributed setting
    using `broadcast_object_list` with CUDA support.

    Parameters:
    obj (Any): The object to synchronize. Can be any picklable object (e.g., list, dict, custom class).
    rank (int): The rank of the device from which to broadcast the object state. Default is 0.

    Note: Ensure torch.distributed is initialized before using this function and CUDA is available.
    """

    # Move the object to a list for broadcasting
    object_list = [obj]

    # Broadcast the object list from the source rank to all other ranks
    dist.broadcast_object_list(object_list, src=rank, device="cuda")

    # Retrieve the synchronized object
    obj = object_list[0]

    return obj

In [ ]:
def format_numel_str(numel: int) -> str:
    """
    Format a number of elements to a human-readable string.

    Args:
        numel (int): The number of elements.

    Returns:
        str: The formatted string.
    """
    B = 1024**3
    M = 1024**2
    K = 1024
    if numel >= B:
        return f"{numel / B:.2f} B"
    elif numel >= M:
        return f"{numel / M:.2f} M"
    elif numel >= K:
        return f"{numel / K:.2f} K"
    else:
        return f"{numel}"

In [ ]:
class VariableVideoBatchSampler(DistributedSampler):
    def __init__(
        self,
        dataset: VideoTextDataset,
        bucket_config: dict,
        num_replicas: int | None = None,
        rank: int | None = None,
        shuffle: bool = True,
        seed: int = 0,
        drop_last: bool = False,
        verbose: bool = False,
        num_bucket_build_workers: int = 1,
        num_groups: int = 1,
    ) -> None:
        super().__init__(
            dataset=dataset, num_replicas=num_replicas, rank=rank, shuffle=shuffle, seed=seed, drop_last=drop_last
        )
        self.dataset = dataset
        assert dataset.bucket_class == "Bucket", "Only support Bucket class for now"
        self.bucket = Bucket(bucket_config)
        self.verbose = verbose
        self.last_micro_batch_access_index = 0
        self.num_bucket_build_workers = num_bucket_build_workers
        self._cached_bucket_sample_dict = None
        self._cached_num_total_batch = None
        self.num_groups = num_groups

        if dist.get_rank() == 0:
            pandarallel.initialize(
                nb_workers=self.num_bucket_build_workers,
                progress_bar=False,
                verbose=0,
                use_memory_fs=False,
            )

    def __iter__(self) -> Iterator[list[int]]:
        bucket_sample_dict, _ = self.group_by_bucket()
        self.clear_cache()

        g = torch.Generator()
        g.manual_seed(self.seed + self.epoch)
        bucket_micro_batch_count = OrderedDict()
        bucket_last_consumed = OrderedDict()

        # process the samples
        for bucket_id, data_list in bucket_sample_dict.items():
            # handle droplast
            bs_per_gpu = self.bucket.get_batch_size(bucket_id)
            remainder = len(data_list) % bs_per_gpu

            if remainder > 0:
                if not self.drop_last:
                    # if there is remainder, we pad to make it divisible
                    data_list += data_list[: bs_per_gpu - remainder]
                else:
                    # we just drop the remainder to make it divisible
                    data_list = data_list[:-remainder]
            bucket_sample_dict[bucket_id] = data_list

            # handle shuffle
            if self.shuffle:
                data_indices = torch.randperm(len(data_list), generator=g).tolist()
                data_list = [data_list[i] for i in data_indices]
                bucket_sample_dict[bucket_id] = data_list

            # compute how many micro-batches each bucket has
            num_micro_batches = len(data_list) // bs_per_gpu
            bucket_micro_batch_count[bucket_id] = num_micro_batches

        # compute the bucket access order
        # each bucket may have more than one batch of data
        # thus bucket_id may appear more than 1 time
        bucket_id_access_order = []
        for bucket_id, num_micro_batch in bucket_micro_batch_count.items():
            bucket_id_access_order.extend([bucket_id] * num_micro_batch)

        # randomize the access order
        if self.shuffle:
            bucket_id_access_order_indices = torch.randperm(len(bucket_id_access_order), generator=g).tolist()
            bucket_id_access_order = [bucket_id_access_order[i] for i in bucket_id_access_order_indices]

        # make the number of bucket accesses divisible by dp size
        remainder = len(bucket_id_access_order) % self.num_replicas
        if remainder > 0:
            if self.drop_last:
                bucket_id_access_order = bucket_id_access_order[: len(bucket_id_access_order) - remainder]
            else:
                bucket_id_access_order += bucket_id_access_order[: self.num_replicas - remainder]

        # prepare each batch from its bucket
        # according to the predefined bucket access order
        num_iters = len(bucket_id_access_order) // self.num_replicas
        start_iter_idx = self.last_micro_batch_access_index // self.num_replicas

        # re-compute the micro-batch consumption
        # this is useful when resuming from a state dict with a different number of GPUs
        self.last_micro_batch_access_index = start_iter_idx * self.num_replicas
        for i in range(self.last_micro_batch_access_index):
            bucket_id = bucket_id_access_order[i]
            bucket_bs = self.bucket.get_batch_size(bucket_id)
            if bucket_id in bucket_last_consumed:
                bucket_last_consumed[bucket_id] += bucket_bs
            else:
                bucket_last_consumed[bucket_id] = bucket_bs

        for i in range(start_iter_idx, num_iters):
            bucket_access_list = bucket_id_access_order[i * self.num_replicas : (i + 1) * self.num_replicas]
            self.last_micro_batch_access_index += self.num_replicas

            # compute the data samples consumed by each access
            bucket_access_boundaries = []
            for bucket_id in bucket_access_list:
                bucket_bs = self.bucket.get_batch_size(bucket_id)
                last_consumed_index = bucket_last_consumed.get(bucket_id, 0)
                bucket_access_boundaries.append([last_consumed_index, last_consumed_index + bucket_bs])

                # update consumption
                if bucket_id in bucket_last_consumed:
                    bucket_last_consumed[bucket_id] += bucket_bs
                else:
                    bucket_last_consumed[bucket_id] = bucket_bs

            # compute the range of data accessed by each GPU
            bucket_id = bucket_access_list[self.rank]
            boundary = bucket_access_boundaries[self.rank]
            cur_micro_batch = bucket_sample_dict[bucket_id][boundary[0] : boundary[1]]

            # encode t, h, w into the sample index
            real_t, real_h, real_w = self.bucket.get_thw(bucket_id)
            cur_micro_batch = [f"{idx}-{real_t}-{real_h}-{real_w}" for idx in cur_micro_batch]
            yield cur_micro_batch

        self.reset()

    def __len__(self) -> int:
        return self.get_num_batch() // self.num_groups

    def get_num_batch(self) -> int:
        _, num_total_batch = self.group_by_bucket()
        return num_total_batch

    def clear_cache(self):
        self._cached_bucket_sample_dict = None
        self._cached_num_total_batch = 0

    def group_by_bucket(self) -> dict:
        """
        Group the dataset samples into buckets.
        This method will set `self._cached_bucket_sample_dict` to the bucket sample dict.

        Returns:
            dict: a dictionary with bucket id as key and a list of sample indices as value
        """
        if self._cached_bucket_sample_dict is not None:
            return self._cached_bucket_sample_dict, self._cached_num_total_batch

        # use pandarallel to accelerate bucket processing
        log_message("Building buckets using %d workers...", self.num_bucket_build_workers)
        bucket_ids = None
        if dist.get_rank() == 0:
            data = self.dataset.data.copy(deep=True)
            data["id"] = data.index
            bucket_ids = data.parallel_apply(
                apply,
                axis=1,
                method=self.bucket.get_bucket_id,
                seed=self.seed + self.epoch,
                num_bucket=self.bucket.num_bucket,
                fps_max=self.dataset.fps_max,
            )
        dist.barrier()
        bucket_ids = sync_object_across_devices(bucket_ids)
        dist.barrier()

        # group by bucket
        # each data sample is put into a bucket with a similar image/video size
        bucket_sample_dict = defaultdict(list)
        bucket_ids_np = np.array(bucket_ids)
        valid_indices = np.where(bucket_ids_np != None)[0]
        for i in valid_indices:
            bucket_sample_dict[bucket_ids_np[i]].append(i)

        # cache the bucket sample dict
        self._cached_bucket_sample_dict = bucket_sample_dict

        # num total batch
        num_total_batch = self.print_bucket_info(bucket_sample_dict)
        self._cached_num_total_batch = num_total_batch

        return bucket_sample_dict, num_total_batch

    def print_bucket_info(self, bucket_sample_dict: dict) -> int:
        # collect statistics
        num_total_samples = num_total_batch = 0
        num_total_img_samples = num_total_vid_samples = 0
        num_total_img_batch = num_total_vid_batch = 0
        num_total_vid_batch_256 = num_total_vid_batch_768 = 0
        num_aspect_dict = defaultdict(lambda: [0, 0])
        num_hwt_dict = defaultdict(lambda: [0, 0])
        for k, v in bucket_sample_dict.items():
            size = len(v)
            num_batch = size // self.bucket.get_batch_size(k[:-1])

            num_total_samples += size
            num_total_batch += num_batch

            if k[1] == 1:
                num_total_img_samples += size
                num_total_img_batch += num_batch
            else:
                if k[0] == "256px":
                    num_total_vid_batch_256 += num_batch
                elif k[0] == "768px":
                    num_total_vid_batch_768 += num_batch
                num_total_vid_samples += size
                num_total_vid_batch += num_batch

            num_aspect_dict[k[-1]][0] += size
            num_aspect_dict[k[-1]][1] += num_batch
            num_hwt_dict[k[:-1]][0] += size
            num_hwt_dict[k[:-1]][1] += num_batch

        # sort
        num_aspect_dict = dict(sorted(num_aspect_dict.items(), key=lambda x: x[0]))
        num_hwt_dict = dict(
            sorted(num_hwt_dict.items(), key=lambda x: (get_num_pexels_from_name(x[0][0]), x[0][1]), reverse=True)
        )
        num_hwt_img_dict = {k: v for k, v in num_hwt_dict.items() if k[1] == 1}
        num_hwt_vid_dict = {k: v for k, v in num_hwt_dict.items() if k[1] > 1}

        # log
        if dist.get_rank() == 0 and self.verbose:
            log_message("Bucket Info:")
            log_message("Bucket [#sample, #batch] by aspect ratio:")
            for k, v in num_aspect_dict.items():
                log_message("(%s): #sample: %s, #batch: %s", k, format_numel_str(v[0]), format_numel_str(v[1]))
            log_message("===== Image Info =====")
            log_message("Image Bucket by HxWxT:")
            for k, v in num_hwt_img_dict.items():
                log_message("%s: #sample: %s, #batch: %s", k, format_numel_str(v[0]), format_numel_str(v[1]))
            log_message("--------------------------------")
            log_message(
                "#image sample: %s, #image batch: %s",
                format_numel_str(num_total_img_samples),
                format_numel_str(num_total_img_batch),
            )
            log_message("===== Video Info =====")
            log_message("Video Bucket by HxWxT:")
            for k, v in num_hwt_vid_dict.items():
                log_message("%s: #sample: %s, #batch: %s", k, format_numel_str(v[0]), format_numel_str(v[1]))
            log_message("--------------------------------")
            log_message(
                "#video sample: %s, #video batch: %s",
                format_numel_str(num_total_vid_samples),
                format_numel_str(num_total_vid_batch),
            )
            log_message("===== Summary =====")
            log_message("#non-empty buckets: %s", len(bucket_sample_dict))
            log_message(
                "Img/Vid sample ratio: %.2f",
                num_total_img_samples / num_total_vid_samples if num_total_vid_samples > 0 else 0,
            )
            log_message(
                "Img/Vid batch ratio: %.2f", num_total_img_batch / num_total_vid_batch if num_total_vid_batch > 0 else 0
            )
            log_message(
                "vid batch 256: %s, vid batch 768: %s", format_numel_str(num_total_vid_batch_256), format_numel_str(num_total_vid_batch_768)
            )
            log_message(
                "Vid batch ratio (256px/768px): %.2f", num_total_vid_batch_256 / num_total_vid_batch_768 if num_total_vid_batch_768 > 0 else 0
            )
            log_message(
                "#training sample: %s, #training batch: %s",
                format_numel_str(num_total_samples),
                format_numel_str(num_total_batch),
            )
        return num_total_batch

    def reset(self):
        self.last_micro_batch_access_index = 0

    def set_step(self, start_step: int):
        self.last_micro_batch_access_index = start_step * self.num_replicas

    def state_dict(self, num_steps: int) -> dict:
        # the last_micro_batch_access_index in the __iter__ is often
        # not accurate during multi-workers and data prefetching
        # thus, we need the user to pass the actual steps which have been executed
        # to calculate the correct last_micro_batch_access_index
        return {"seed": self.seed, "epoch": self.epoch, "last_micro_batch_access_index": num_steps * self.num_replicas}

    def load_state_dict(self, state_dict: dict) -> None:
        self.__dict__.update(state_dict)

In [ ]:
def collate_fn_default(batch):
    # filter out None
    batch = [x for x in batch if x is not None]
    assert len(batch) > 0, "batch is empty"

    # HACK: for loading text features
    use_mask = False
    if "mask" in batch[0] and isinstance(batch[0]["mask"], int):
        masks = [x.pop("mask") for x in batch]

        texts = [x.pop("text") for x in batch]
        texts = torch.cat(texts, dim=1)
        use_mask = True

    ret = torch.utils.data.default_collate(batch)

    if use_mask:
        ret["mask"] = masks
        ret["text"] = texts
    return ret

In [ ]:

# Deterministic dataloader
def get_seed_worker(seed):
    def seed_worker(worker_id):
        worker_seed = seed
        if seed is not None:
            np.random.seed(worker_seed)
            torch.manual_seed(worker_seed)
            random.seed(worker_seed)

    return seed_worker

In [ ]:
import collections
import functools
import os
import queue
import random
import threading

import numpy as np
import torch
import torch.multiprocessing as multiprocessing
from torch._utils import ExceptionWrapper
from torch.distributed import ProcessGroup
from torch.utils.data import DataLoader, _utils
from torch.utils.data._utils import MP_STATUS_CHECK_INTERVAL
from torch.utils.data.dataloader import (
    IterDataPipe,
    MapDataPipe,
    _BaseDataLoaderIter,
    _MultiProcessingDataLoaderIter,
    _sharding_worker_init_fn,
    _SingleProcessDataLoaderIter,
)

In [ ]:
def _pin_memory_loop(
    in_queue, out_queue, device_id, done_event, device, pin_memory_cache: PinMemoryCache, pin_memory_key: str
):
    # This setting is thread local, and prevents the copy in pin_memory from
    # consuming all CPU cores.
    torch.set_num_threads(1)

    if device == "cuda":
        torch.cuda.set_device(device_id)
    elif device == "xpu":
        torch.xpu.set_device(device_id)  # type: ignore[attr-defined]
    elif device == torch._C._get_privateuse1_backend_name():
        custom_device_mod = getattr(torch, torch._C._get_privateuse1_backend_name())
        custom_device_mod.set_device(device_id)

    def do_one_step():
        try:
            r = in_queue.get(timeout=MP_STATUS_CHECK_INTERVAL)
        except queue.Empty:
            return
        idx, data = r
        if not done_event.is_set() and not isinstance(data, ExceptionWrapper):
            try:
                assert isinstance(data, dict)
                if pin_memory_key in data:
                    val = data[pin_memory_key]
                    pin_memory_value = pin_memory_cache.get(val)
                    pin_memory_value.copy_(val)
                    data[pin_memory_key] = pin_memory_value
            except Exception:
                data = ExceptionWrapper(where=f"in pin memory thread for device {device_id}")
            r = (idx, data)
        while not done_event.is_set():
            try:
                out_queue.put(r, timeout=MP_STATUS_CHECK_INTERVAL)
                break
            except queue.Full:
                continue

    # See NOTE [ Data Loader Multiprocessing Shutdown Logic ] for details on the
    # logic of this function.
    while not done_event.is_set():
        # Make sure that we don't preserve any object from one iteration
        # to the next
        do_one_step()

In [ ]:
class _MultiProcessingDataLoaderIterForVideo(_MultiProcessingDataLoaderIter):
    pin_memory_key: str = "video"

    def __init__(self, loader):
        _BaseDataLoaderIter.__init__(self, loader)
        self.pin_memory_cache = PinMemoryCache()

        self._prefetch_factor = loader.prefetch_factor

        assert self._num_workers > 0
        assert self._prefetch_factor > 0

        if loader.multiprocessing_context is None:
            multiprocessing_context = multiprocessing
        else:
            multiprocessing_context = loader.multiprocessing_context

        self._worker_init_fn = loader.worker_init_fn

        # Adds forward compatibilities so classic DataLoader can work with DataPipes:
        #   Additional worker init function will take care of sharding in MP and Distributed
        if isinstance(self._dataset, (IterDataPipe, MapDataPipe)):
            self._worker_init_fn = functools.partial(
                _sharding_worker_init_fn, self._worker_init_fn, self._world_size, self._rank
            )

        # No certainty which module multiprocessing_context is
        self._worker_result_queue = multiprocessing_context.Queue()  # type: ignore[var-annotated]
        self._worker_pids_set = False
        self._shutdown = False
        self._workers_done_event = multiprocessing_context.Event()

        self._index_queues = []
        self._workers = []
        for i in range(self._num_workers):
            # No certainty which module multiprocessing_context is
            index_queue = multiprocessing_context.Queue()  # type: ignore[var-annotated]
            # Need to `cancel_join_thread` here!
            # See sections (2) and (3b) above.
            index_queue.cancel_join_thread()
            w = multiprocessing_context.Process(
                target=_utils.worker._worker_loop,
                args=(
                    self._dataset_kind,
                    self._dataset,
                    index_queue,
                    self._worker_result_queue,
                    self._workers_done_event,
                    self._auto_collation,
                    self._collate_fn,
                    self._drop_last,
                    self._base_seed,
                    self._worker_init_fn,
                    i,
                    self._num_workers,
                    self._persistent_workers,
                    self._shared_seed,
                ),
            )
            w.daemon = True
            # NB: Process.start() actually take some time as it needs to
            #     start a process and pass the arguments over via a pipe.
            #     Therefore, we only add a worker to self._workers list after
            #     it started, so that we do not call .join() if program dies
            #     before it starts, and __del__ tries to join but will get:
            #     AssertionError: can only join a started process.
            w.start()
            self._index_queues.append(index_queue)
            self._workers.append(w)

        if self._pin_memory:
            self._pin_memory_thread_done_event = threading.Event()

            # Queue is not type-annotated
            self._data_queue = queue.Queue()  # type: ignore[var-annotated]
            if self._pin_memory_device == "xpu":
                current_device = torch.xpu.current_device()  # type: ignore[attr-defined]
            elif self._pin_memory_device == torch._C._get_privateuse1_backend_name():
                custom_device_mod = getattr(torch, torch._C._get_privateuse1_backend_name())
                current_device = custom_device_mod.current_device()
            else:
                current_device = torch.cuda.current_device()  # choose cuda for default
            pin_memory_thread = threading.Thread(
                target=_pin_memory_loop,
                args=(
                    self._worker_result_queue,
                    self._data_queue,
                    current_device,
                    self._pin_memory_thread_done_event,
                    self._pin_memory_device,
                    self.pin_memory_cache,
                    self.pin_memory_key,
                ),
            )
            pin_memory_thread.daemon = True
            pin_memory_thread.start()
            # Similar to workers (see comment above), we only register
            # pin_memory_thread once it is started.
            self._pin_memory_thread = pin_memory_thread
        else:
            self._data_queue = self._worker_result_queue  # type: ignore[assignment]

        # In some rare cases, persistent workers (daemonic processes)
        # would be terminated before `__del__` of iterator is invoked
        # when main process exits
        # It would cause failure when pin_memory_thread tries to read
        # corrupted data from worker_result_queue
        # atexit is used to shutdown thread and child processes in the
        # right sequence before main process exits
        if self._persistent_workers and self._pin_memory:
            import atexit

            for w in self._workers:
                atexit.register(_MultiProcessingDataLoaderIter._clean_up_worker, w)

        # .pid can be None only before process is spawned (not the case, so ignore)
        _utils.signal_handling._set_worker_pids(id(self), tuple(w.pid for w in self._workers))  # type: ignore[misc]
        _utils.signal_handling._set_SIGCHLD_handler()
        self._worker_pids_set = True
        self._reset(loader, first_iter=True)

    def remove_cache(self, output_tensor: torch.Tensor):
        self.pin_memory_cache.remove(output_tensor)

    def get_cache_info(self) -> str:
        return str(self.pin_memory_cache)

In [ ]:
class DataloaderForVideo(DataLoader):
    def _get_iterator(self) -> "_BaseDataLoaderIter":
        if self.num_workers == 0:
            return _SingleProcessDataLoaderIter(self)
        else:
            self.check_worker_number_rationality()
            return _MultiProcessingDataLoaderIterForVideo(self)

In [ ]:
def prepare_dataloader(
    dataset,
    batch_size=None,
    shuffle=False,
    seed=1024,
    drop_last=False,
    pin_memory=False,
    num_workers=0,
    process_group: ProcessGroup | None = None,
    bucket_config=None,
    num_bucket_build_workers=1,
    prefetch_factor=None,
    cache_pin_memory=False,
    num_groups=1,
    **kwargs,
):
    _kwargs = kwargs.copy()
    if isinstance(dataset, VideoTextDataset):
        batch_sampler = VariableVideoBatchSampler(
            dataset,
            bucket_config,
            num_replicas=process_group.size(),
            rank=process_group.rank(),
            shuffle=shuffle,
            seed=seed,
            drop_last=drop_last,
            verbose=True,
            num_bucket_build_workers=num_bucket_build_workers,
            num_groups=num_groups,
        )
        dl_cls = DataloaderForVideo if cache_pin_memory else DataLoader
        return (
            dl_cls(
                dataset,
                batch_sampler=batch_sampler,
                worker_init_fn=get_seed_worker(seed),
                pin_memory=pin_memory,
                num_workers=num_workers,
                collate_fn=collate_fn_default,
                prefetch_factor=prefetch_factor,
                **_kwargs,
            ),
            batch_sampler,
        )
    elif isinstance(dataset, TextDataset):
        if process_group is None:
            return (
                DataLoader(
                    dataset,
                    batch_size=batch_size,
                    shuffle=shuffle,
                    worker_init_fn=get_seed_worker(seed),
                    drop_last=drop_last,
                    pin_memory=pin_memory,
                    num_workers=num_workers,
                    prefetch_factor=prefetch_factor,
                    **_kwargs,
                ),
                None,
            )
        else:
            sampler = DistributedSampler(
                dataset,
                num_replicas=process_group.size(),
                rank=process_group.rank(),
                shuffle=shuffle,
                seed=seed,
                drop_last=drop_last,
            )
            return (
                DataLoader(
                    dataset,
                    sampler=sampler,
                    worker_init_fn=get_seed_worker(seed),
                    pin_memory=pin_memory,
                    num_workers=num_workers,
                    collate_fn=collate_fn_default,
                    prefetch_factor=prefetch_factor,
                    **_kwargs,
                ),
                sampler,
            )
    else:
        raise ValueError(f"Unsupported dataset type: {type(dataset)}")

In [ ]:
# == build dataloader ==
dataloader, sampler = prepare_dataloader(
    bucket_config=cfg.get("bucket_config", None),
    num_bucket_build_workers=cfg.get("num_bucket_build_workers", 1),
    **dataloader_args,
)
dataloader, sampler

In [ ]:
import math
import os
import random
import re
from typing import Any

import numpy as np
import pandas as pd
import requests
import torch
import torch.distributed as dist
import torchvision
import torchvision.transforms as transforms
from PIL import Image
from torchvision.datasets.folder import IMG_EXTENSIONS, pil_loader
from torchvision.io import write_video
from torchvision.utils import save_image

In [ ]:
# first_batch = next(iter(dataloader))
# first_batch

In [ ]:
# first_batch = next(iter(dataloader))
# first_batch["sampling_interval"], first_batch["video"].shape, first_batch["path"], first_batch["index"]

In [ ]:
# first_frame = first_batch["video"][:, :, 0].to(device=device, dtype=torch.float32)
# first_frame.shape

# import matplotlib.pyplot as plt
# plt.imshow(first_frame[0].permute(1, 2, 0).cpu().numpy())
# plt.axis("off")
# plt.show()

## モデル構築

In [ ]:
cfg.model

In [ ]:
from typing import Callable, Optional

import diffusers
import torch
from huggingface_hub import PyTorchModelHubMixin
from torch import nn

# from opensora.registry import MODELS
# from opensora.utils.ckpt import load_checkpoint

from opensora.models.dc_ae.ae_model_zoo import DCAE_HF
from opensora.models.dc_ae.models.dc_ae import DCAE, DCAEConfig, dc_ae_f32

In [ ]:
# @MODELS.register_module("dc_ae")
def DC_AE(
    model_name: str,
    device_map: str | torch.device = "cuda",
    torch_dtype: torch.dtype = torch.bfloat16,
    from_scratch: bool = False,
    from_pretrained: str | None = None,
    is_training: bool = False,
    use_spatial_tiling: bool = False,
    use_temporal_tiling: bool = False,
    spatial_tile_size: int = 256,
    temporal_tile_size: int = 32,
    tile_overlap_factor: float = 0.25,
    scaling_factor: float = None,
    disc_off_grad_ckpt: bool = False,
) -> DCAE_HF:
    if not from_scratch:
        model = DCAE_HF.from_pretrained(model_name).to(device_map, torch_dtype)
    else:
        model = DCAE_HF(model_name).to(device_map, torch_dtype)

    if from_pretrained is not None:
        model = load_checkpoint(model, from_pretrained, device_map=device_map)
        print(f"loaded dc_ae from ckpt path: {from_pretrained}")

    model.cfg.is_training = is_training
    model.use_spatial_tiling = use_spatial_tiling
    model.use_temporal_tiling = use_temporal_tiling
    model.spatial_tile_size = spatial_tile_size
    model.temporal_tile_size = temporal_tile_size
    model.tile_overlap_factor = tile_overlap_factor
    if scaling_factor is not None:
        model.scaling_factor = scaling_factor
    model.decoder.disc_off_grad_ckpt = disc_off_grad_ckpt
    return model

# {'type': 'dc_ae',
#  'model_name': 'dc-ae-f32t4c128',
#  'from_scratch': True,
#  'from_pretrained': None}

model = DC_AE(
    model_name=cfg.model.model_name,
    from_scratch=cfg.model.get("from_scratch", False),
    from_pretrained=cfg.model.get("from_pretrained", None),
)
model

In [ ]:

def get_model_numel(model: torch.nn.Module) -> tuple[int, int]:
    """
    Get the number of parameters in a model.

    Args:
        model (torch.nn.Module): The model.

    Returns:
        tuple[int, int]: The total number of parameters and the number of trainable parameters.
    """
    num_params = 0
    num_params_trainable = 0
    for p in model.parameters():
        num_params += p.numel()
        if p.requires_grad:
            num_params_trainable += p.numel()
    return num_params, num_params_trainable

In [ ]:
def log_model_params(model: nn.Module):
    """
    Log the number of parameters in a model.

    Args:
        model (torch.nn.Module): The model.
    """
    num_params, num_params_trainable = get_model_numel(model)
    model_name = model.__class__.__name__
    log_message(f"[{model_name}] Number of parameters: {format_numel_str(num_params)}")
    log_message(f"[{model_name}] Number of trainable parameters: {format_numel_str(num_params_trainable)}")


log_model_params(model)

In [ ]:
# False
if cfg.get("grad_checkpoint", False):
    set_grad_checkpoint(model)

### 損失

In [ ]:
from torchvision import models

In [ ]:
class vgg16(torch.nn.Module):
    def __init__(self, requires_grad=False, pretrained=True):
        super(vgg16, self).__init__()
        vgg_pretrained_features = models.vgg16(pretrained=pretrained).features
        self.slice1 = torch.nn.Sequential()
        self.slice2 = torch.nn.Sequential()
        self.slice3 = torch.nn.Sequential()
        self.slice4 = torch.nn.Sequential()
        self.slice5 = torch.nn.Sequential()
        self.N_slices = 5
        for x in range(4):
            self.slice1.add_module(str(x), vgg_pretrained_features[x])
        for x in range(4, 9):
            self.slice2.add_module(str(x), vgg_pretrained_features[x])
        for x in range(9, 16):
            self.slice3.add_module(str(x), vgg_pretrained_features[x])
        for x in range(16, 23):
            self.slice4.add_module(str(x), vgg_pretrained_features[x])
        for x in range(23, 30):
            self.slice5.add_module(str(x), vgg_pretrained_features[x])
        if not requires_grad:
            for param in self.parameters():
                param.requires_grad = False

    def forward(self, X):
        h = self.slice1(X)
        h_relu1_2 = h
        h = self.slice2(h)
        h_relu2_2 = h
        h = self.slice3(h)
        h_relu3_3 = h
        h = self.slice4(h)
        h_relu4_3 = h
        h = self.slice5(h)
        h_relu5_3 = h
        vgg_outputs = namedtuple("VggOutputs", ["relu1_2", "relu2_2", "relu3_3", "relu4_3", "relu5_3"])
        out = vgg_outputs(h_relu1_2, h_relu2_2, h_relu3_3, h_relu4_3, h_relu5_3)
        return out

In [ ]:
class NetLinLayer(nn.Module):
    """A single linear layer which does a 1x1 conv"""

    def __init__(self, chn_in, chn_out=1, use_dropout=False):
        super(NetLinLayer, self).__init__()
        layers = (
            [
                nn.Dropout(),
            ]
            if (use_dropout)
            else []
        )
        layers += [
            nn.Conv2d(chn_in, chn_out, 1, stride=1, padding=0, bias=False),
        ]
        self.model = nn.Sequential(*layers)

In [ ]:

URL_MAP = {"vgg_lpips": "https://heibox.uni-heidelberg.de/f/607503859c864bc1b30b/?dl=1"}

CKPT_MAP = {"vgg_lpips": "vgg.pth"}

MD5_MAP = {"vgg_lpips": "d507d7349b931f0638a25a48a722f98a"}


In [ ]:
def get_ckpt_path(name, root=".", check=False):
    assert name in URL_MAP
    path = os.path.join(root, CKPT_MAP[name])
    if not os.path.exists(path) or (check and not md5_hash(path) == MD5_MAP[name]):
        print("Downloading {} model from {} to {}".format(name, URL_MAP[name], path))
        download(URL_MAP[name], path)
        md5 = md5_hash(path)
        assert md5 == MD5_MAP[name], md5
    return path

In [ ]:
import warnings
from collections.abc import Iterable
from typing import Callable, ContextManager, Optional, Tuple

import torch
import torch.nn as nn
from colossalai.utils import get_current_device
from torch.utils.checkpoint import (
    _DEFAULT_DETERMINISM_MODE,
    CheckpointFunction,
    _checkpoint_without_reentrant_generator,
    checkpoint_sequential,
    noop_context_fn,
)

In [ ]:
# TorchDynamo does not step inside utils.checkpoint function.  The flow
# looks likes this
#  1) TorchDynamo tries to wrap utils.checkpoint in a HigherOrderOp by
#     speculatively checking if the forward function is safe to trace.
#  2) If yes, then Dynamo-generated Fx graph has the wrapped higher
#     order op. As a result, TorchDynamo does not look inside utils.checkpoint.
#  3) If not, then TorchDynamo falls back to eager by performing a graph
#     break. And here, the following disable wrapper ensures that
#     TorchDynamo does not trigger again on the frames created by
#     utils.checkpoint innards.
@torch._disable_dynamo
def checkpoint(
    function,
    *args,
    use_reentrant: Optional[bool] = None,
    context_fn: Callable[[], Tuple[ContextManager, ContextManager]] = noop_context_fn,
    determinism_check: str = _DEFAULT_DETERMINISM_MODE,
    debug: bool = False,
    **kwargs,
):
    r"""Checkpoint a model or part of the model.

    Activation checkpointing is a technique that trades compute for memory.
    Instead of keeping tensors needed for backward alive until they are used in
    gradient computation during backward, forward computation in checkpointed
    regions omits saving tensors for backward and recomputes them during the
    backward pass. Activation checkpointing can be applied to any part of a
    model.

    There are currently two checkpointing implementations available, determined
    by the :attr:`use_reentrant` parameter. It is recommended that you use
    ``use_reentrant=False``. Please refer the note below for a discussion of
    their differences.

    .. warning::

        If the :attr:`function` invocation during the backward pass differs
        from the forward pass, e.g., due to a global variable, the checkpointed
        version may not be equivalent, potentially causing an
        error being raised or leading to silently incorrect gradients.

    .. warning::

        The ``use_reentrant`` parameter should be passed explicitly. In version
        2.4 we will raise an exception if ``use_reentrant`` is not passed.
        If you are using the ``use_reentrant=True`` variant, please refer to the
        note below for important considerations and potential limitations.

    .. note::

        The reentrant variant of checkpoint (``use_reentrant=True``) and
        the non-reentrant variant of checkpoint (``use_reentrant=False``)
        differ in the following ways:

        * Non-reentrant checkpoint stops recomputation as soon as all needed
          intermediate activations have been recomputed. This feature is enabled
          by default, but can be disabled with :func:`set_checkpoint_early_stop`.
          Reentrant checkpoint always recomputes :attr:`function` in its
          entirety during the backward pass.

        * The reentrant variant does not record the autograd graph during the
          forward pass, as it runs with the forward pass under
          :func:`torch.no_grad`. The non-reentrant version does record the
          autograd graph, allowing one to perform backward on the graph within
          checkpointed regions.

        * The reentrant checkpoint only supports the
          :func:`torch.autograd.backward` API for the backward pass without its
          `inputs` argument, while the non-reentrant version supports all ways
          of performing the backward pass.

        * At least one input and output must have ``requires_grad=True`` for the
          reentrant variant. If this condition is unmet, the checkpointed part
          of the model will not have gradients. The non-reentrant version does
          not have this requirement.

        * The reentrant version does not consider tensors in nested structures
          (e.g., custom objects, lists, dicts, etc) as participating in
          autograd, while the non-reentrant version does.

        * The reentrant checkpoint does not support checkpointed regions with
          detached tensors from the computational graph, whereas the
          non-reentrant version does. For the reentrant variant, if the
          checkpointed segment contains tensors detached using ``detach()`` or
          with :func:`torch.no_grad`, the backward pass will raise an error.
          This is because ``checkpoint`` makes all the outputs require gradients
          and this causes issues when a tensor is defined to have no gradient in
          the model. To avoid this, detach the tensors outside of the
          ``checkpoint`` function.

    Args:
        function: describes what to run in the forward pass of the model or
            part of the model. It should also know how to handle the inputs
            passed as the tuple. For example, in LSTM, if user passes
            ``(activation, hidden)``, :attr:`function` should correctly use the
            first input as ``activation`` and the second input as ``hidden``
        preserve_rng_state(bool, optional):  Omit stashing and restoring
            the RNG state during each checkpoint. Note that under torch.compile,
            this flag doesn't take effect and we always preserve RNG state.
            Default: ``True``
        use_reentrant(bool):
            specify whether to use the activation checkpoint variant that
            requires reentrant autograd. This parameter should be passed
            explicitly. In version 2.4 we will raise an exception if
            ``use_reentrant`` is not passed. If ``use_reentrant=False``,
            ``checkpoint`` will use an implementation that does not require
            reentrant autograd. This allows ``checkpoint`` to support additional
            functionality, such as working as expected with
            ``torch.autograd.grad`` and support for keyword arguments input into
            the checkpointed function.
        context_fn(Callable, optional): A callable returning a tuple of two
            context managers. The function and its recomputation will be run
            under the first and second context managers respectively.
            This argument is only supported if ``use_reentrant=False``.
        determinism_check(str, optional): A string specifying the determinism
            check to perform. By default it is set to ``"default"`` which
            compares the shapes, dtypes, and devices of the recomputed tensors
            against those the saved tensors. To turn off this check, specify
            ``"none"``. Currently these are the only two supported values.
            Please open an issue if you would like to see more determinism
            checks. This argument is only supported if ``use_reentrant=False``,
            if ``use_reentrant=True``, the determinism check is always disabled.
        debug(bool, optional): If ``True``, error messages will also include
            a trace of the operators ran during the original forward computation
            as well as the recomputation. This argument is only supported if
            ``use_reentrant=False``.
        args: tuple containing inputs to the :attr:`function`

    Returns:
        Output of running :attr:`function` on :attr:`*args`
    """
    if use_reentrant is None:
        warnings.warn(
            "torch.utils.checkpoint: the use_reentrant parameter should be "
            "passed explicitly. In version 2.4 we will raise an exception "
            "if use_reentrant is not passed. use_reentrant=False is "
            "recommended, but if you need to preserve the current default "
            "behavior, you can pass use_reentrant=True. Refer to docs for more "
            "details on the differences between the two variants.",
            stacklevel=2,
        )
        use_reentrant = True

    # Hack to mix *args with **kwargs in a python 2.7-compliant way
    preserve = kwargs.pop("preserve_rng_state", True)
    if kwargs and use_reentrant:
        raise ValueError("Unexpected keyword arguments: " + ",".join(arg for arg in kwargs))

    if use_reentrant:
        if context_fn is not noop_context_fn or debug is not False:
            raise ValueError("Passing `context_fn` or `debug` is only supported when " "use_reentrant=False.")
        return CheckpointFunctionWithOffload.apply(function, preserve, *args)
    else:
        gen = _checkpoint_without_reentrant_generator(
            function, preserve, context_fn, determinism_check, debug, *args, **kwargs
        )
        # Runs pre-forward logic
        next(gen)
        ret = function(*args, **kwargs)
        # Runs post-forward logic
        try:
            next(gen)
        except StopIteration:
            return ret

In [ ]:
def normalize_tensor(x, eps=1e-10):
    norm_factor = torch.sqrt(torch.sum(x**2, dim=1, keepdim=True))
    return x / (norm_factor + eps)

In [ ]:
def spatial_average(x, keepdim=True):
    return x.mean([2, 3], keepdim=keepdim)

In [ ]:
class LPIPS(nn.Module):
    # Learned perceptual metric
    def __init__(self, use_dropout=True):
        super().__init__()
        self.scaling_layer = ScalingLayer()
        self.chns = [64, 128, 256, 512, 512]  # vg16 features
        self.net = vgg16(pretrained=True, requires_grad=False)
        self.lin0 = NetLinLayer(self.chns[0], use_dropout=use_dropout)
        self.lin1 = NetLinLayer(self.chns[1], use_dropout=use_dropout)
        self.lin2 = NetLinLayer(self.chns[2], use_dropout=use_dropout)
        self.lin3 = NetLinLayer(self.chns[3], use_dropout=use_dropout)
        self.lin4 = NetLinLayer(self.chns[4], use_dropout=use_dropout)
        self.lins = [self.lin0, self.lin1, self.lin2, self.lin3, self.lin4]
        self.load_from_pretrained()
        for param in self.parameters():
            param.requires_grad = False

    def load_from_pretrained(self, name="vgg_lpips"):
        path = os.path.expanduser("~/.cache/opensora/taming/modules/autoencoder/lpips")
        ckpt = get_ckpt_path(name, path)
        self.load_state_dict(torch.load(ckpt, map_location=torch.device("cpu")), strict=False)

    @classmethod
    def from_pretrained(cls, name="vgg_lpips"):
        if name != "vgg_lpips":
            raise NotImplementedError
        model = cls()
        ckpt = get_ckpt_path(name)
        model.load_state_dict(torch.load(ckpt, map_location=torch.device("cpu")), strict=False)
        return model

    def forward_old(self, input, target):
        in0_input, in1_input = (self.scaling_layer(input), self.scaling_layer(target))
        outs0, outs1 = self.net(in0_input), self.net(in1_input)
        feats0, feats1, diffs = {}, {}, {}
        lins = [self.lin0, self.lin1, self.lin2, self.lin3, self.lin4]
        for kk in range(len(self.chns)):
            feats0[kk], feats1[kk] = normalize_tensor(outs0[kk]), normalize_tensor(outs1[kk])
            diffs[kk] = (feats0[kk] - feats1[kk]) ** 2

        res = [spatial_average(lins[kk].model(diffs[kk]), keepdim=True) for kk in range(len(self.chns))]
        val = res[0]
        for l in range(1, len(self.chns)):
            val += res[l]
        return val

    def get_layer_loss(self, input, target, i):
        input, target = getattr(self.net, f"slice{i+1}")(input), getattr(self.net, f"slice{i+1}")(target)
        feats0, feats1 = normalize_tensor(input), normalize_tensor(target)
        diff = (feats0 - feats1) ** 2
        avg = spatial_average(self.lins[i].model(diff), keepdim=True)
        return avg, input, target

    def forward(self, input, target):
        input, target = (self.scaling_layer(input), self.scaling_layer(target))

        val = None
        for i in range(len(self.chns)):
            avg, input, target = checkpoint(self.get_layer_loss, input, target, i, use_reentrant=False)
            val = avg if val is None else val + avg
        return val

In [ ]:
class ScalingLayer(nn.Module):
    def __init__(self):
        super(ScalingLayer, self).__init__()
        self.register_buffer("shift", torch.Tensor([-0.030, -0.088, -0.188])[None, :, None, None])
        self.register_buffer("scale", torch.Tensor([0.458, 0.448, 0.450])[None, :, None, None])

    def forward(self, inp):
        return (inp - self.shift) / self.scale

In [ ]:
def l1(x, y):
    return torch.abs(x - y)

In [ ]:
def batch_mean(x):
    return torch.sum(x) / x.shape[0]

In [ ]:
class VAELoss(nn.Module):
    def __init__(
        self,
        logvar_init=0.0,
        perceptual_loss_weight=1.0,
        kl_loss_weight=5e-4,
        device="cpu",
        dtype="bf16",
    ):
        super().__init__()

        if type(dtype) == str:
            if dtype == "bf16":
                dtype = torch.bfloat16
            elif dtype == "fp16":
                dtype = torch.float16
            elif dtype == "fp32":
                dtype = torch.float32
            else:
                raise NotImplementedError(f"dtype: {dtype}")

        # KL Loss
        self.kl_weight = kl_loss_weight
        # Perceptual Loss
        self.perceptual_loss_fn = LPIPS().eval().to(device, dtype)
        self.perceptual_loss_fn.requires_grad_(False)
        self.perceptual_loss_weight = perceptual_loss_weight
        self.logvar = nn.Parameter(torch.ones(size=()) * logvar_init)

    def forward(
        self,
        video,
        recon_video,
        posterior,
    ) -> dict:
        video.size(0)
        video = rearrange(video, "b c t h w -> (b t) c h w").contiguous()
        recon_video = rearrange(recon_video, "b c t h w -> (b t) c h w").contiguous()

        # reconstruction loss
        recon_loss = l1(video, recon_video)

        # perceptual loss
        perceptual_loss = self.perceptual_loss_fn(video, recon_video)
        # nll loss (from reconstruction loss and perceptual loss)
        nll_loss = recon_loss + perceptual_loss * self.perceptual_loss_weight
        nll_loss = nll_loss / torch.exp(self.logvar) + self.logvar

        # Batch Mean
        nll_loss = batch_mean(nll_loss)
        recon_loss = batch_mean(recon_loss)
        numel_elements = video.numel() // video.size(0)
        perceptual_loss = batch_mean(perceptual_loss) * numel_elements

        # KL Loss
        if posterior is None:
            kl_loss = torch.tensor(0.0).to(video.device, video.dtype)
        else:
            kl_loss = posterior.kl()
            kl_loss = batch_mean(kl_loss)
        weighted_kl_loss = kl_loss * self.kl_weight

        return {
            "nll_loss": nll_loss,
            "kl_loss": weighted_kl_loss,
            "recon_loss": recon_loss,
            "perceptual_loss": perceptual_loss,
        }

In [ ]:
vae_loss_fn = VAELoss(**cfg.vae_loss_config, device=device, dtype=dtype)

### EMAモデルの構築

In [ ]:
from copy import deepcopy

In [ ]:
def record_model_param_shape(model: torch.nn.Module) -> dict:
    """
    Record the shape of the model parameters.

    Args:
        model (torch.nn.Module): The model to record the parameter shape of.

    Returns:
        dict: The shape of the model parameters.
    """
    param_shape = {}
    for name, param in model.named_parameters():
        param_shape[name] = param.shape
    return param_shape


In [ ]:
# == build EMA model ==
if cfg.get("ema_decay", None) is not None:
    ema = deepcopy(model).cpu().eval().requires_grad_(False)
    ema_shape_dict = record_model_param_shape(ema)
    logger.info("EMA model created.")
else:
    ema = ema_shape_dict = None
    logger.info("No EMA model created.")

### ディスクリミネーターモデルの作成

In [ ]:
# == build discriminator model ==
use_discriminator = cfg.get("discriminator", None) is not None
if use_discriminator:
    discriminator = build_module(cfg.discriminator, MODELS).to(device, dtype).train()
    log_model_params(discriminator)
    generator_loss_fn = GeneratorLoss(**cfg.gen_loss_config)
    discriminator_loss_fn = DiscriminatorLoss(**cfg.disc_loss_config)

### オプティマイザの作成

In [ ]:
from opensora.utils.optimizer import create_lr_scheduler, create_optimizer

In [ ]:
# == setup optimizer ==
optimizer = create_optimizer(model, cfg.optim)

### 訓練スケジューラの作成

In [ ]:
num_steps_per_epoch = len(dataloader)

In [ ]:
# == setup lr scheduler ==
lr_scheduler = create_lr_scheduler(
    optimizer=optimizer, num_steps_per_epoch=num_steps_per_epoch, epochs=cfg.get("epochs", 1000), **cfg.lr_scheduler
)

### ディスクリミネーターオプティマイザ

In [ ]:
# == setup discriminator optimizer ==
if use_discriminator:
    disc_optimizer = create_optimizer(discriminator, cfg.optim_discriminator)
    disc_lr_scheduler = create_lr_scheduler(
        optimizer=disc_optimizer,
        num_steps_per_epoch=num_steps_per_epoch,
        epochs=cfg.get("epochs", 1000),
        **cfg.disc_lr_scheduler,
    )

### 訓練の準備

In [ ]:
# =======================================================
# 4. distributed training preparation with colossalai
# =======================================================
logger.info("Preparing for distributed training...")
# == boosting ==
torch.set_default_dtype(dtype)
model, optimizer, _, dataloader, lr_scheduler = booster.boost(
    model=model,
    optimizer=optimizer,
    lr_scheduler=lr_scheduler,
    dataloader=dataloader,
)

In [ ]:
# False

if use_discriminator:
    discriminator, disc_optimizer, _, _, disc_lr_scheduler = booster.boost(
        model=discriminator,
        optimizer=disc_optimizer,
        lr_scheduler=disc_lr_scheduler,
    )

In [ ]:
torch.set_default_dtype(torch.float)
logger.info("Boosted model for distributed training")

In [ ]:
def all_reduce_sum(tensor: torch.Tensor) -> torch.Tensor:
    dist.all_reduce(tensor=tensor, group=get_data_parallel_group())
    return tensor

In [ ]:
# == global variables ==
cfg_epochs = cfg.get("epochs", 1000)
mixed_strategy = cfg.get("mixed_strategy", None)
mixed_image_ratio = cfg.get("mixed_image_ratio", 0.0)
# modulate mixed image ratio since we force rank 0 to be video
num_ranks = dist.get_world_size()
modulated_mixed_image_ratio = (
    num_ranks * mixed_image_ratio / (num_ranks - 1) if num_ranks > 1 else mixed_image_ratio
)
if is_log_process(plugin_type, plugin_config):
    print("modulated mixed image ratio:", modulated_mixed_image_ratio)

start_epoch = start_step = log_step = acc_step = 0
running_loss = dict(  # loss accumulated over config.log_every steps
    all=0.0,
    nll=0.0,
    nll_rec=0.0,
    nll_per=0.0,
    kl=0.0,
    gen=0.0,
    gen_w=0.0,
    disc=0.0,
    debug=0.0,
)

def log_loss(name, loss, loss_dict, use_video):
    # only calculate loss for video
    if use_video == 0:
        loss.data = torch.tensor(0.0, device=device, dtype=dtype)
    all_reduce_sum(loss.data)
    num_video = torch.tensor(use_video, device=device, dtype=dtype)
    all_reduce_sum(num_video)
    loss_item = loss.item() / num_video.item()
    loss_dict[name] = loss_item
    running_loss[name] += loss_item

logger.info("Training for %s epochs with %s steps per epoch", cfg_epochs, num_steps_per_epoch)

### EMAモデルをシャーディング

In [ ]:

def model_sharding(model: torch.nn.Module, device: torch.device = None):
    """
    Sharding the model parameters across multiple GPUs.

    Args:
        model (torch.nn.Module): The model to shard.
        device (torch.device): The device to shard the model to.
    """
    global_rank = dist.get_rank()
    world_size = dist.get_world_size()
    for _, param in model.named_parameters():
        if device is None:
            device = param.device
        padding_size = (world_size - param.numel() % world_size) % world_size
        if padding_size > 0:
            padding_param = torch.nn.functional.pad(param.data.view(-1), [0, padding_size])
        else:
            padding_param = param.data.view(-1)
        splited_params = padding_param.split(padding_param.numel() // world_size)
        splited_params = splited_params[global_rank]
        param.data = splited_params.to(device)

In [ ]:
# == sharding EMA model ==
if ema is not None:
    model_sharding(ema)
    ema = ema.to(device)

if cfg.get("freeze_layers", None) == "all":
    for param in model.module.parameters():
        param.requires_grad = False
    print("all layers frozen")

# model.module.requires_grad_(False)

### 再開

In [ ]:
# == resume ==
if cfg.get("load", None) is not None:
    logger.info("Loading checkpoint from %s", cfg.load)
    start_epoch = cfg.get("start_epoch", None)
    start_step = cfg.get("start_step", None)
    ret = checkpoint_io.load(
        booster,
        cfg.load,
        model=model,
        ema=ema,
        optimizer=optimizer,
        lr_scheduler=lr_scheduler,
        sampler=(
            None if start_step is not None else sampler
        ),  # if specify start step, set last_micro_batch_access_index of a new sampler instead
    )
    if start_step is not None:
        # if start step exceeds data length, go to next epoch
        if start_step > num_steps_per_epoch:
            start_epoch = (
                start_epoch + start_step // num_steps_per_epoch
                if start_epoch is not None
                else start_step // num_steps_per_epoch
            )
            start_step = start_step % num_steps_per_epoch
        sampler.set_step(start_step)

    start_epoch = start_epoch if start_epoch is not None else ret[0]
    start_step = start_step if start_step is not None else ret[1]

    if (
        use_discriminator
        and os.path.exists(os.path.join(cfg.load, "discriminator"))
        and not cfg.get("restart_disc", False)
    ):
        booster.load_model(discriminator, os.path.join(cfg.load, "discriminator"))
        if cfg.get("load_optimizer", True):
            booster.load_optimizer(disc_optimizer, os.path.join(cfg.load, "disc_optimizer"))
            if disc_lr_scheduler is not None:
                booster.load_lr_scheduler(disc_lr_scheduler, os.path.join(cfg.load, "disc_lr_scheduler"))
            if cfg.get("disc_lr", None) is not None:
                set_lr(disc_optimizer, disc_lr_scheduler, cfg.disc_lr)

    logger.info("Loaded checkpoint %s at epoch %s step %s", cfg.load, start_epoch, start_step)

    if cfg.get("lr", None) is not None:
        set_lr(optimizer, lr_scheduler, cfg.lr, cfg.get("initial_lr", None))

    if cfg.get("update_warmup_steps", False):
        assert (
            cfg.lr_scheduler.get("warmup_steps", None) is not None
        ), "you need to set lr_scheduler.warmup_steps in order to pass --update-warmup-steps True"
        set_warmup_steps(lr_scheduler, cfg.lr_scheduler.warmup_steps)
        if use_discriminator:
            assert (
                cfg.disc_lr_scheduler.get("warmup_steps", None) is not None
            ), "you need to set disc_lr_scheduler.warmup_steps in order to pass --update-warmup-steps True"
            set_warmup_steps(disc_lr_scheduler, cfg.disc_lr_scheduler.warmup_steps)

## 訓練ループ

In [ ]:
import gc
import os
import random
import subprocess
import warnings
from contextlib import nullcontext
from copy import deepcopy
from pprint import pformat

warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=UserWarning)
gc.disable()


import torch
import torch.distributed as dist
from colossalai.booster import Booster
from colossalai.utils import set_seed
from torch.profiler import ProfilerActivity, profile, schedule
from tqdm import tqdm

import wandb

In [ ]:
# =======================================================
# 5. training loop
# =======================================================
dist.barrier()

cfg["accumulation_steps"] = 128
accumulation_steps = int(cfg.get("accumulation_steps", 1))
logger.info("Using gradient accumulation steps: %s", accumulation_steps)

In [ ]:
for epoch in range(start_epoch, cfg_epochs):
    # == set dataloader to new epoch ==
    sampler.set_epoch(epoch)
    dataiter = iter(dataloader)
    logger.info("Beginning epoch %s...", epoch)
    random.seed(1024 + dist.get_rank())  # load vid/img for each rank

    # == training loop in an epoch ==
    with tqdm(
        enumerate(dataiter, start=start_step),
        desc=f"Epoch {epoch}",
        disable=not coordinator.is_master(),
        total=num_steps_per_epoch,
        initial=start_step,
    ) as pbar:
        pbar_iter = iter(pbar)

        def fetch_data():
            step, batch = next(pbar_iter)
            pinned_video = batch["video"]
            batch["video"] = pinned_video.to(device, dtype, non_blocking=True)
            return batch, step, pinned_video

        batch_, step_, pinned_video_ = fetch_data()

        profiler_ctxt = (
            profile(
                activities=[ProfilerActivity.CPU, ProfilerActivity.CUDA],
                schedule=my_schedule,
                on_trace_ready=torch.profiler.tensorboard_trace_handler("./log/profile"),
                record_shapes=True,
                profile_memory=True,
                with_stack=True,
            )
            if cfg.get("profile", False)
            else nullcontext()
        )

        with profiler_ctxt:
            for _ in range(start_step, num_steps_per_epoch):
                if cfg.get("profile", False) and _ == WARMUP + ACTIVE + WAIT + 3:
                    break

                # == load data ===
                batch, step, pinned_video = batch_, step_, pinned_video_
                if step + 1 < num_steps_per_epoch:
                    batch_, step_, pinned_video_ = fetch_data()

                # == log config ==
                global_step = epoch * num_steps_per_epoch + step
                actual_update_step = (global_step + 1) // accumulation_steps
                log_step += 1
                acc_step += 1

                # == mixed strategy ==
                x = batch["video"]
                t_length = x.size(2)
                use_video = 1
                if mixed_strategy == "mixed_video_image":
                    if random.random() < modulated_mixed_image_ratio and dist.get_rank() != 0:
                        # NOTE: enable the first rank to use video
                        t_length = 1
                        use_video = 0
                elif mixed_strategy == "mixed_video_random":
                    t_length = random.randint(1, x.size(2))
                x = x[:, :, :t_length, :, :]

                with Timer("model", log=True) if cfg.get("profile", False) else nullcontext():
                    # == forward pass ==
                    x_rec, posterior, z = model(x)

                    if cfg.get("profile", False):
                        profiler_ctxt.step()

                    if cache_pin_memory:
                        dataiter.remove_cache(pinned_video)

                    # == loss initialization ==
                    vae_loss = torch.tensor(0.0, device=device, dtype=dtype)
                    loss_dict = {}  # loss at every step

                    # == reconstruction loss ==
                    ret = vae_loss_fn(x, x_rec, posterior)
                    nll_loss = ret["nll_loss"]
                    kl_loss = ret["kl_loss"]
                    recon_loss = ret["recon_loss"]
                    perceptual_loss = ret["perceptual_loss"]
                    vae_loss += nll_loss + kl_loss

                    # == generator loss ==
                    if use_discriminator:
                        # turn off grad update for disc
                        discriminator.requires_grad_(False)
                        fake_logits = discriminator(x_rec.contiguous())

                        generator_loss, g_loss = generator_loss_fn(
                            fake_logits,
                            nll_loss,
                            model.module.get_last_layer(),
                            actual_update_step,
                            is_training=model.training,
                        )
                        # print(f"generator_loss: {generator_loss}, recon_loss: {recon_loss}, perceptual_loss: {perceptual_loss}")

                        vae_loss += generator_loss
                        # turn on disc training
                        discriminator.requires_grad_(True)

                    # == generator backward & update ==
                    ctx = (
                        booster.no_sync(model, optimizer)
                        if cfg.get("plugin", "zero2") in ("zero1", "zero1-seq")
                        and (step + 1) % accumulation_steps != 0
                        else nullcontext()
                    )
                    with Timer("backward", log=True) if cfg.get("profile", False) else nullcontext():
                        with ctx:
                            booster.backward(loss=vae_loss / accumulation_steps, optimizer=optimizer)

                    with Timer("optimizer", log=True) if cfg.get("profile", False) else nullcontext():
                        if (step + 1) % accumulation_steps == 0:
                            optimizer.step()
                            optimizer.zero_grad()
                            if lr_scheduler is not None:
                                lr_scheduler.step(
                                    actual_update_step,
                                )
                            # == update EMA ==
                            if ema is not None:
                                update_ema(
                                    ema,
                                    model.unwrap(),
                                    optimizer=optimizer,
                                    decay=cfg.get("ema_decay", 0.9999),
                                )

                # == logging ==
                log_loss("all", vae_loss, loss_dict, use_video)
                log_loss("nll", nll_loss, loss_dict, use_video)
                log_loss("nll_rec", recon_loss, loss_dict, use_video)
                log_loss("nll_per", perceptual_loss, loss_dict, use_video)
                log_loss("kl", kl_loss, loss_dict, use_video)
                if use_discriminator:
                    log_loss("gen_w", generator_loss, loss_dict, use_video)
                    log_loss("gen", g_loss, loss_dict, use_video)

                # == loss: discriminator adversarial ==
                if use_discriminator:
                    real_logits = discriminator(x.detach().contiguous())
                    fake_logits = discriminator(x_rec.detach().contiguous())
                    disc_loss = discriminator_loss_fn(
                        real_logits,
                        fake_logits,
                        actual_update_step,
                    )

                    # == discriminator backward & update ==
                    ctx = (
                        booster.no_sync(discriminator, disc_optimizer)
                        if cfg.get("plugin", "zero2") in ("zero1", "zero1-seq")
                        and (step + 1) % accumulation_steps != 0
                        else nullcontext()
                    )
                    with ctx:
                        booster.backward(loss=disc_loss / accumulation_steps, optimizer=disc_optimizer)
                    if (step + 1) % accumulation_steps == 0:
                        disc_optimizer.step()
                        disc_optimizer.zero_grad()
                        if disc_lr_scheduler is not None:
                            disc_lr_scheduler.step(actual_update_step)

                    # log
                    log_loss("disc", disc_loss, loss_dict, use_video)

                # == logging ==
                if (global_step + 1) % accumulation_steps == 0:
                    if coordinator.is_master() and actual_update_step % cfg.get("log_every", 1) == 0:
                        avg_loss = {k: v / log_step for k, v in running_loss.items()}
                        # progress bar
                        pbar.set_postfix(
                            {
                                # "step": step,
                                # "global_step": global_step,
                                # "actual_update_step": actual_update_step,
                                # "lr": optimizer.param_groups[0]["lr"],
                                **{k: f"{v:.2f}" for k, v in avg_loss.items()},
                            }
                        )
                        # tensorboard
                        tb_writer.add_scalar("loss", vae_loss.item(), actual_update_step)
                        # wandb
                        if cfg.get("wandb", False):
                            wandb.log(
                                {
                                    "iter": global_step,
                                    "epoch": epoch,
                                    "lr": optimizer.param_groups[0]["lr"],
                                    "avg_loss_": avg_loss,
                                    "avg_loss": avg_loss["all"],
                                    "loss_": loss_dict,
                                    "loss": vae_loss.item(),
                                    "global_grad_norm": optimizer.get_grad_norm(),
                                },
                                step=actual_update_step,
                            )

                        running_loss = {k: 0.0 for k in running_loss}
                        log_step = 0

                    # == checkpoint saving ==
                    ckpt_every = cfg.get("ckpt_every", 0)
                    if ckpt_every > 0 and actual_update_step % ckpt_every == 0 and coordinator.is_master():
                        subprocess.run("sudo drop_cache", shell=True)

                    if ckpt_every > 0 and actual_update_step % ckpt_every == 0:
                        # mannually garbage collection
                        gc.collect()

                        save_dir = checkpoint_io.save(
                            booster,
                            exp_dir,
                            model=model,
                            ema=ema,
                            optimizer=optimizer,
                            lr_scheduler=lr_scheduler,
                            sampler=sampler,
                            epoch=epoch,
                            step=step + 1,
                            global_step=global_step + 1,
                            batch_size=cfg.get("batch_size", None),
                            actual_update_step=actual_update_step,
                            ema_shape_dict=ema_shape_dict,
                            async_io=True,
                        )

                        if is_log_process(plugin_type, plugin_config):
                            os.system(f"chgrp -R share {save_dir}")

                        if use_discriminator:
                            booster.save_model(discriminator, os.path.join(save_dir, "discriminator"), shard=True)
                            booster.save_optimizer(
                                disc_optimizer,
                                os.path.join(save_dir, "disc_optimizer"),
                                shard=True,
                                size_per_shard=4096,
                            )
                            if disc_lr_scheduler is not None:
                                booster.save_lr_scheduler(
                                    disc_lr_scheduler, os.path.join(save_dir, "disc_lr_scheduler")
                                )
                        dist.barrier()

                        logger.info(
                            "Saved checkpoint at epoch %s, step %s, global_step %s to %s",
                            epoch,
                            step + 1,
                            actual_update_step,
                            save_dir,
                        )

                        # remove old checkpoints
                        rm_checkpoints(exp_dir, keep_n_latest=cfg.get("keep_n_latest", -1))
                        logger.info(
                            "Removed old checkpoints and kept %s latest ones.", cfg.get("keep_n_latest", -1)
                        )

        if cfg.get("profile", False):
            profiler_ctxt.export_chrome_trace("./log/profile/trace.json")

    sampler.reset()
    start_step = 0